# Análise dos dados das variáveis relacionadas ao IDEB

O notebook contém experimentos com essas variáveis. São utilizadas duas bases de dados:<br>
- Váriáveis derivadas do SAEB (informações socioeconômicas)<br> 
- Variáveis com informações das escolas e professores<br>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, RFECV, SelectFromModel
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy import array 
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

## Importando bases de dados

In [ ]:
#Importando variáveis derivadas do SAEB

#base_estudantes_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_se.csv')

base_estudantes_se = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_se.csv')

#base_estudantes_se.set_index('ID_ESCOLA', inplace=True)

#base_estudantes_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_al.csv')

#base_estudantes_al = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_al.csv')

#base_estudantes_al.set_index('ID_ESCOLA', inplace=True)

#base_estudantes_ba = pd.read_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_ba.csv')

dados_pessoais = base_estudantes_se

#base_estudantes_ba.set_index('ID_ESCOLA', inplace=True)

#dados_pessoais = pd.merge(base_estudantes_se, base_estudantes_al, how = 'outer')

#dados_pessoais = pd.merge(base_estudantes_ba, dados_pessoais, how = 'outer')

In [ ]:
dados_pessoais.set_index('ID_ESCOLA', inplace=True)

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28012631,2017,2,28,2800100,2,2,1,12,6,15,4,16,3,15,4,6,13,19,0,13,6,14,5,18,1,13,6,1,18,4,15,15,3,9,9,17,1,1,12,7,...,1,0,1,6,5,5,2,1,6,1,0,4,4,3,0,2,1,5,2,2,7,0,0,9,6,3,1,0,0,1,2,3,1,1,0,0,3,3,2,1
61178208,2017,2,28,2800209,2,2,1,19,32,39,12,33,18,34,16,33,16,49,1,41,9,38,12,48,2,32,18,9,40,8,39,38,8,25,21,41,8,1,38,10,...,4,0,8,6,18,11,8,6,19,8,7,3,2,6,4,15,5,8,7,0,11,0,2,10,15,9,5,4,6,2,2,3,5,6,4,8,4,6,5,2
61211437,2017,2,28,2800209,2,2,1,41,64,80,25,65,37,71,34,48,57,103,3,94,12,87,18,101,3,65,40,18,87,18,85,89,17,45,57,90,13,1,65,36,...,4,1,12,23,30,20,18,6,35,16,10,15,8,14,13,22,17,11,11,4,28,0,2,30,35,13,11,8,6,6,9,12,9,9,9,10,9,5,10,7
28017846,2017,2,28,2800308,1,2,1,21,20,37,4,31,10,35,6,21,18,39,2,35,6,37,4,39,2,29,12,11,30,6,35,31,10,30,10,36,4,0,22,18,...,3,0,3,13,17,6,2,1,4,7,9,15,1,4,0,10,8,5,9,0,9,0,1,12,12,10,6,0,0,1,4,10,2,7,4,1,4,3,3,1
28017854,2017,2,28,2800308,1,2,1,39,62,93,10,77,25,92,10,60,41,100,2,92,9,86,14,95,5,60,40,37,65,15,80,85,15,55,44,76,19,8,48,52,...,6,2,9,21,28,25,18,3,20,17,17,32,1,13,5,12,19,17,24,3,24,0,2,22,36,17,14,5,6,8,7,9,4,10,13,5,15,11,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61247893,2017,2,28,6307258,2,4,1,18,28,47,0,43,4,48,1,35,13,47,2,45,4,47,2,49,0,38,11,3,45,2,46,38,11,16,32,45,1,1,38,10,...,0,0,4,14,21,7,1,0,0,2,9,23,9,4,0,0,4,9,21,5,9,0,3,33,8,3,0,0,0,2,7,3,6,4,3,4,4,5,3,5
61229418,2017,2,28,6307267,2,4,1,27,31,58,1,52,6,54,5,43,14,59,1,56,4,54,6,55,2,47,12,8,51,12,45,44,13,34,26,52,5,2,37,20,...,1,0,6,16,20,10,7,0,2,10,11,20,7,9,0,7,11,8,17,1,15,0,0,29,29,0,0,1,0,10,4,5,4,3,4,4,4,6,4,5
61214852,2017,2,28,6307284,2,4,1,10,9,18,1,14,5,19,0,10,8,18,1,14,5,15,4,19,0,10,9,3,16,2,17,15,4,9,10,16,2,1,14,5,...,3,0,6,3,7,3,0,0,2,1,3,6,4,3,0,0,4,5,2,3,5,0,3,10,2,2,0,2,0,2,1,4,1,0,0,3,3,0,2,3


**Pré-processamento da base de dados pessoais**

In [ ]:
#Função que verifica se o ID de uma escola é inválido
#Segundo o Dicionário de Dados da base o SAEB 6 - Os códigos dos Municípios e 
#das Escolas que começam com o dígito "6" são máscaras, isto é, são códigos fictícios.
def retira_id_ficticio(id_escola):
    if id_escola[0] == "6":
        return True
    else:
        return False

In [ ]:
#Se retirou as escolas fictícias
for escola in dados_pessoais.index:
    if (retira_id_ficticio(str(escola))):
        dados_pessoais.drop(escola, inplace=True)
        

dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28012631,2017,2,28,2800100,2,2,1,12,6,15,4,16,3,15,4,6,13,19,0,13,6,14,5,18,1,13,6,1,18,4,15,15,3,9,9,17,1,1,12,7,...,1,0,1,6,5,5,2,1,6,1,0,4,4,3,0,2,1,5,2,2,7,0,0,9,6,3,1,0,0,1,2,3,1,1,0,0,3,3,2,1
28017846,2017,2,28,2800308,1,2,1,21,20,37,4,31,10,35,6,21,18,39,2,35,6,37,4,39,2,29,12,11,30,6,35,31,10,30,10,36,4,0,22,18,...,3,0,3,13,17,6,2,1,4,7,9,15,1,4,0,10,8,5,9,0,9,0,1,12,12,10,6,0,0,1,4,10,2,7,4,1,4,3,3,1
28017854,2017,2,28,2800308,1,2,1,39,62,93,10,77,25,92,10,60,41,100,2,92,9,86,14,95,5,60,40,37,65,15,80,85,15,55,44,76,19,8,48,52,...,6,2,9,21,28,25,18,3,20,17,17,32,1,13,5,12,19,17,24,3,24,0,2,22,36,17,14,5,6,8,7,9,4,10,13,5,15,11,5,10
28018400,2017,2,28,2800308,1,2,1,36,36,71,2,55,18,65,6,39,31,71,2,59,14,56,16,70,3,47,26,15,56,3,68,54,17,45,27,57,10,5,40,30,...,2,1,12,10,24,14,12,0,8,13,13,25,8,6,3,10,5,17,19,7,12,1,5,34,24,8,1,0,0,8,6,10,8,8,4,4,5,3,6,7
28018435,2017,2,28,2800308,1,2,1,56,79,124,14,101,37,121,20,67,74,136,4,120,20,115,26,135,4,87,52,36,103,14,123,122,19,77,58,105,25,6,64,70,...,6,3,13,36,44,24,18,2,22,17,22,48,14,13,5,19,28,23,30,6,29,0,6,40,43,20,17,6,5,4,13,10,14,21,11,10,11,11,7,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2017,2,28,2807105,2,2,1,106,186,253,39,198,95,199,95,109,182,287,4,265,25,246,44,286,3,193,97,59,230,15,276,218,75,175,117,247,36,9,176,103,...,8,2,43,61,89,52,46,15,104,71,23,43,15,21,43,101,57,18,24,5,46,0,6,95,80,51,31,15,16,23,28,37,24,26,21,24,25,18,22,29
28009487,2017,2,28,2807105,2,2,2,18,23,32,10,31,11,22,19,18,22,39,3,36,6,33,9,41,1,27,15,5,37,2,39,33,8,22,20,35,4,3,29,12,...,2,0,4,9,10,12,7,4,18,9,4,2,1,4,5,17,7,2,2,0,9,0,0,17,8,7,5,4,1,4,3,1,4,6,3,7,2,1,4,4
28014839,2017,2,28,2807204,2,2,1,31,42,68,6,57,17,55,18,41,31,73,1,71,3,66,8,74,0,56,18,7,64,15,57,66,7,39,33,64,9,2,35,35,...,6,0,11,17,16,14,16,2,16,13,14,15,5,10,5,17,12,7,10,2,20,0,3,24,20,10,9,4,4,5,8,5,5,11,5,8,6,5,8,5


In [ ]:
#Importando variáveis com informações das escolas e professores
#base_escola_se = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-SE.csv')

base_escola_se = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-SE.csv')

base_escola_se.set_index('CO_ENTIDADE', inplace=True)

#base_escola_al = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-AL.csv')

#base_escola_al = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-AL.csv')

#base_escola_al.set_index('CO_ENTIDADE', inplace=True)

#base_escola_ba = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-BA.csv')

#base_escola_ba = pd.read_csv('/content/drive/MyDrive/TCC/ESCOLAS2017-BA.csv')

#base_escola_ba.rename(columns={'Unnamed: 0': 'CO_ENTIDADE'}, inplace=True)

#base_escola_ba.set_index('CO_ENTIDADE', inplace=True)

#dados_escola = pd.merge(base_escola_se, base_escola_al, how = 'outer')

#dados_escola = pd.concat([dados_escola, base_escola_ba], join='outer')

dados_escola = base_escola_se

In [ ]:
#dados_escola.set_index('CO_ENTIDADE', inplace=True)

dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0.0,1,16/01/2017,24/11/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0,16,3,0,27,1,45,0,0,0,0,0,45,45,345,45,7,6.9
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2.0,1,01/02/2017,08/12/2017,2,2802,28005,28,2807402,280740205,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,0,5,0,0,19,3,24,0,0,0,0,0,24,24,103,24,4,6.6
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0.0,1,09/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,5,0,0,12,1,17,0,0,0,0,0,17,17,89,17,5,6.4
28032497,2017,COLEGIO NOTA DEZ,3.0,1,23/01/2017,15/12/2017,2,2802,28004,28,2803906,280390605,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,2.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,2,1,0,5,0,11,0,0,0,0,0,11,11,21,11,1,6.1
28017773,2017,COL AMERICANO BATISTA,0.0,1,16/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,6,1,0,15,0,22,0,0,0,0,0,22,22,152,22,6,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28022033,2017,COLEGIO ESTADUAL CLEONICE SOARES DA FONSECA,2.0,1,20/02/2017,21/12/2017,2,2803,28012,28,2800670,280067005,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,25,0,0,12,4,2,11,3,27,0,0,27,0,0,0,27,326,27,12,4.0
28005759,2017,COLEGIO ESTADUAL EMILIANO GUIMARAES,6.0,1,27/03/2017,16/01/2018,2,2802,28003,28,2803807,280380705,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1

In [ ]:
col_ideb = dados_escola['Ideb-2017']

dados_escola.drop('Ideb-2017', axis=1, inplace=True)

In [ ]:
col_ideb

CO_ENTIDADE
28017943    6.9
28010124    6.6
28018052    6.4
28032497    6.1
28017773    5.8
           ... 
28022033    4.0
28005759    3.0
28018940    3.0
28025903    3.0
28029577    3.0
Name: Ideb-2017, Length: 96, dtype: float64

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_CATEGORIA_ESCOLA_PRIVADA,IN_CONVENIADA_PP,TP_CONVENIO_PODER_PUBLICO,IN_MANT_ESCOLA_PRIVADA_EMP,IN_MANT_ESCOLA_PRIVADA_ONG,IN_MANT_ESCOLA_PRIVADA_SIND,IN_MANT_ESCOLA_PRIVADA_SIST_S,IN_MANT_ESCOLA_PRIVADA_S_FINS,CO_ESCOLA_SEDE_VINCULADA,CO_IES_OFERTANTE,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,TP_OCUPACAO_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0.0,1,16/01/2017,24/11/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0,16,3,0,27,1,45,0,0,0,0,0,45,45,345,45,7
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2.0,1,01/02/2017,08/12/2017,2,2802,28005,28,2807402,280740205,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,0,5,0,0,19,3,24,0,0,0,0,0,24,24,103,24,4
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0.0,1,09/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,5,0,0,12,1,17,0,0,0,0,0,17,17,89,17,5
28032497,2017,COLEGIO NOTA DEZ,3.0,1,23/01/2017,15/12/2017,2,2802,28004,28,2803906,280390605,4,1,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,NaN,1,1,2.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,2,1,0,5,0,11,0,0,0,0,0,11,11,21,11,1
28017773,2017,COL AMERICANO BATISTA,0.0,1,16/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1.0,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,6,1,0,15,0,22,0,0,0,0,0,22,22,152,22,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28022033,2017,COLEGIO ESTADUAL CLEONICE SOARES DA FONSECA,2.0,1,20/02/2017,21/12/2017,2,2803,28012,28,2800670,280067005,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,0,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,25,0,0,12,4,2,11,3,27,0,0,27,0,0,0,27,326,27,12
28005759,2017,COLEGIO ESTADUAL EMILIANO GUIMARAES,6.0,1,27/03/2017,16/01/2018,2,2802,28003,28,2803807,280380705,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0,0,0,0,0,0,0,0,NaN,0,0,0.0,1,1,...,0,0,

#### Pré-processamento da base das escolas

In [ ]:
#Retirando valores ausentes
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL                 1
TP_CATEGORIA_ESCOLA_PRIVADA      76
IN_CONVENIADA_PP                 76
TP_CONVENIO_PODER_PUBLICO        96
IN_MANT_ESCOLA_PRIVADA_EMP       76
IN_MANT_ESCOLA_PRIVADA_ONG       76
IN_MANT_ESCOLA_PRIVADA_SIND      76
IN_MANT_ESCOLA_PRIVADA_SIST_S    76
IN_MANT_ESCOLA_PRIVADA_S_FINS    76
CO_ESCOLA_SEDE_VINCULADA         96
CO_IES_OFERTANTE                 96
TP_OCUPACAO_PREDIO_ESCOLAR        1
TP_OCUPACAO_GALPAO               96
IN_PREDIO_COMPARTILHADO           1
IN_FUNDAMENTAL_CICLOS             3
TP_INDIGENA_LINGUA               96
CO_LINGUA_INDIGENA               96
dtype: int64

In [ ]:
#Retirando colunas que apresentam mais de 60% do valores nulos
dados_escola.dropna(thresh=len(dados_escola)*0.6, axis=1, inplace=True)

#Preenchendo valores ausentes
dados_escola.fillna(0, inplace=True)

filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL             0
TP_OCUPACAO_PREDIO_ESCOLAR    0
IN_PREDIO_COMPARTILHADO       0
IN_FUNDAMENTAL_CICLOS         0
dtype: int64

In [ ]:
dados_escola[listOfColumnNames] = dados_escola[listOfColumnNames].astype(int)

In [ ]:
dados_escola.insert(len(dados_escola.columns), 'Ideb-2017', col_ideb)
#['Ideb-2017'] = col_ideb

In [ ]:
dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,IN_AGUA_INEXISTENTE,IN_ENERGIA_REDE_PUBLICA,IN_ENERGIA_GERADOR,IN_ENERGIA_OUTROS,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_ESGOTO_INEXISTENTE,...,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0,1,16/01/2017,24/11/2017,2,2803,28011,28,2800308,280030805,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,0,0,16,3,0,27,1,45,0,0,0,0,0,45,45,345,45,7,6.9
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2,1,01/02/2017,08/12/2017,2,2802,28005,28,2807402,280740205,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0,0,5,0,0,19,3,24,0,0,0,0,0,24,24,103,24,4,6.6
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0,1,09/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16,0,0,5,0,0,12,1,17,0,0,0,0,0,17,17,89,17,5,6.4
28032497,2017,COLEGIO NOTA DEZ,3,1,23/01/2017,15/12/2017,2,2802,28004,28,2803906,280390605,4,1,1,1,2,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,0,2,1,0,5,0,11,0,0,0,0,0,11,11,21,11,1,6.1
28017773,2017,COL AMERICANO BATISTA,0,1,16/01/2017,01/12/2017,2,2803,28011,28,2800308,280030805,4,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,0,...,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,0,0,6,1,0,15,0,22,0,0,0,0,0,22,22,152,22,6,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28022033,2017,COLEGIO ESTADUAL CLEONICE SOARES DA FONSECA,2,1,20/02/2017,21/12/2017,2,2803,28012,28,2800670,280067005,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,...,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,25,0,0,12,4,2,11,3,27,0,0,27,0,0,0,27,326,27,12,4.0
28005759,2017,COLEGIO ESTADUAL EMILIANO GUIMARAES,6,1,27/03/2017,16/01/2018,2,2802,28003,28,2803807,280380705,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,15,0,0,6,0,0,10,0,16,0,0,14,2,0,0,16,85,16,5,3.0
28018940,2017,COLEGIO ESTADUAL SANTOS DUMONT,0,1,02/05/2017,21/03/2018,2,2803,28011,28,2800308,280030805,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0

In [ ]:
#Colunas categóricas
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.object]

listOfColumnNames = list(filteredColumns.index)

dados_escola[listOfColumnNames]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,NO_ENTIDADE,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO
CO_ENTIDADE,,,
28017943,COLEGIO AMADEUS LTDA,16/01/2017,24/11/2017
28010124,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,01/02/2017,08/12/2017
28018052,COLEGIO SANTANNA DE SERGIPE LTDA,09/01/2017,01/12/2017
28032497,COLEGIO NOTA DEZ,23/01/2017,15/12/2017
28017773,COL AMERICANO BATISTA,16/01/2017,01/12/2017
...,...,...,...
28022033,COLEGIO ESTADUAL CLEONICE SOARES DA FONSECA,20/02/2017,21/12/2017
28005759,COLEGIO ESTADUAL EMILIANO GUIMARAES,27/03/2017,16/01/2018
28018940,COLEGIO ESTADUAL SANTOS DUMONT,02/05/2017,21/03/2018


In [ ]:
#Tratando datas, convertendo para timestamp

inicio = pd.to_datetime(dados_escola['DT_ANO_LETIVO_INICIO'], infer_datetime_format = True)
termino = pd.to_datetime(dados_escola['DT_ANO_LETIVO_TERMINO'], infer_datetime_format = True)

dados_escola['DT_ANO_LETIVO_INICIO'] = inicio.values.astype (np.int64)
dados_escola['DT_ANO_LETIVO_TERMINO'] = termino.values.astype (np.int64)

In [ ]:
dados_escola.drop('NO_ENTIDADE', axis=1, inplace=True)

In [ ]:
dados_escola.columns.to_list()

['NU_ANO_CENSO',
 'CO_ORGAO_REGIONAL',
 'TP_SITUACAO_FUNCIONAMENTO',
 'DT_ANO_LETIVO_INICIO',
 'DT_ANO_LETIVO_TERMINO',
 'CO_REGIAO',
 'CO_MESORREGIAO',
 'CO_MICRORREGIAO',
 'CO_UF',
 'CO_MUNICIPIO',
 'CO_DISTRITO',
 'TP_DEPENDENCIA',
 'TP_LOCALIZACAO',
 'TP_REGULAMENTACAO',
 'IN_LOCAL_FUNC_PREDIO_ESCOLAR',
 'TP_OCUPACAO_PREDIO_ESCOLAR',
 'IN_LOCAL_FUNC_SALAS_EMPRESA',
 'IN_LOCAL_FUNC_SOCIOEDUCATIVO',
 'IN_LOCAL_FUNC_UNID_PRISIONAL',
 'IN_LOCAL_FUNC_PRISIONAL_SOCIO',
 'IN_LOCAL_FUNC_TEMPLO_IGREJA',
 'IN_LOCAL_FUNC_CASA_PROFESSOR',
 'IN_LOCAL_FUNC_GALPAO',
 'IN_LOCAL_FUNC_SALAS_OUTRA_ESC',
 'IN_LOCAL_FUNC_OUTROS',
 'IN_PREDIO_COMPARTILHADO',
 'IN_AGUA_FILTRADA',
 'IN_AGUA_REDE_PUBLICA',
 'IN_AGUA_POCO_ARTESIANO',
 'IN_AGUA_CACIMBA',
 'IN_AGUA_FONTE_RIO',
 'IN_AGUA_INEXISTENTE',
 'IN_ENERGIA_REDE_PUBLICA',
 'IN_ENERGIA_GERADOR',
 'IN_ENERGIA_OUTROS',
 'IN_ENERGIA_INEXISTENTE',
 'IN_ESGOTO_REDE_PUBLICA',
 'IN_ESGOTO_FOSSA',
 'IN_ESGOTO_INEXISTENTE',
 'IN_LIXO_COLETA_PERIODICA',
 'IN_LIXO_

In [ ]:
dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q016_B,TX_RESP_Q016_C,TX_RESP_Q016_D,TX_RESP_Q016_E,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_B,TX_RESP_Q019_C,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_A,TX_RESP_Q023_B,TX_RESP_Q023_C,TX_RESP_Q023_D,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q004_A,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_D,TX_RESP_Q004_E,TX_RESP_Q004_F,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_A,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28012631,2017,2,28,2800100,2,2,1,12,6,15,4,16,3,15,4,6,13,19,0,13,6,14,5,18,1,13,6,1,18,4,15,15,3,9,9,17,1,1,12,7,...,1,0,1,6,5,5,2,1,6,1,0,4,4,3,0,2,1,5,2,2,7,0,0,9,6,3,1,0,0,1,2,3,1,1,0,0,3,3,2,1
28017846,2017,2,28,2800308,1,2,1,21,20,37,4,31,10,35,6,21,18,39,2,35,6,37,4,39,2,29,12,11,30,6,35,31,10,30,10,36,4,0,22,18,...,3,0,3,13,17,6,2,1,4,7,9,15,1,4,0,10,8,5,9,0,9,0,1,12,12,10,6,0,0,1,4,10,2,7,4,1,4,3,3,1
28017854,2017,2,28,2800308,1,2,1,39,62,93,10,77,25,92,10,60,41,100,2,92,9,86,14,95,5,60,40,37,65,15,80,85,15,55,44,76,19,8,48,52,...,6,2,9,21,28,25,18,3,20,17,17,32,1,13,5,12,19,17,24,3,24,0,2,22,36,17,14,5,6,8,7,9,4,10,13,5,15,11,5,10
28018400,2017,2,28,2800308,1,2,1,36,36,71,2,55,18,65,6,39,31,71,2,59,14,56,16,70,3,47,26,15,56,3,68,54,17,45,27,57,10,5,40,30,...,2,1,12,10,24,14,12,0,8,13,13,25,8,6,3,10,5,17,19,7,12,1,5,34,24,8,1,0,0,8,6,10,8,8,4,4,5,3,6,7
28018435,2017,2,28,2800308,1,2,1,56,79,124,14,101,37,121,20,67,74,136,4,120,20,115,26,135,4,87,52,36,103,14,123,122,19,77,58,105,25,6,64,70,...,6,3,13,36,44,24,18,2,22,17,22,48,14,13,5,19,28,23,30,6,29,0,6,40,43,20,17,6,5,4,13,10,14,21,11,10,11,11,7,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2017,2,28,2807105,2,2,1,106,186,253,39,198,95,199,95,109,182,287,4,265,25,246,44,286,3,193,97,59,230,15,276,218,75,175,117,247,36,9,176,103,...,8,2,43,61,89,52,46,15,104,71,23,43,15,21,43,101,57,18,24,5,46,0,6,95,80,51,31,15,16,23,28,37,24,26,21,24,25,18,22,29
28009487,2017,2,28,2807105,2,2,2,18,23,32,10,31,11,22,19,18,22,39,3,36,6,33,9,41,1,27,15,5,37,2,39,33,8,22,20,35,4,3,29,12,...,2,0,4,9,10,12,7,4,18,9,4,2,1,4,5,17,7,2,2,0,9,0,0,17,8,7,5,4,1,4,3,1,4,6,3,7,2,1,4,4
28014839,2017,2,28,2807204,2,2,1,31,42,68,6,57,17,55,18,41,31,73,1,71,3,66,8,74,0,56,18,7,64,15,57,66,7,39,33,64,9,2,35,35,...,6,0,11,17,16,14,16,2,16,13,14,15,5,10,5,17,12,7,10,2,20,0,3,24,20,10,9,4,4,5,8,5,5,11,5,8,6,5,8,5


### Unindo as bases de dados 

Foi realizado join nos dois datasets, nesse momento se percebeu que não existem variáveis escolas particulares na base derivada do SAEB

In [ ]:
dataset = dados_pessoais.join(dados_escola, how="inner")

In [ ]:
dataset

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,Ideb-2017
28012631,2017,2,28,2800100,2,2,1,12,6,15,4,16,3,15,4,6,13,19,0,13,6,14,5,18,1,13,6,1,18,4,15,15,3,9,9,17,1,1,12,7,...,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,8,0,0,4,1,0,3,0,8,0,0,7,1,0,0,8,22,8,2,3.1
28017846,2017,2,28,2800308,1,2,1,21,20,37,4,31,10,35,6,21,18,39,2,35,6,37,4,39,2,29,12,11,30,6,35,31,10,30,10,36,4,0,22,18,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,10,2,0,10,0,22,0,0,22,0,0,0,22,121,22,5,3.9
28017854,2017,2,28,2800308,1,2,1,39,62,93,10,77,25,92,10,60,41,100,2,92,9,86,14,95,5,60,40,37,65,15,80,85,15,55,44,76,19,8,48,52,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,0,0,17,2,0,13,0,32,0,0,32,0,0,0,32,301,32,9,2.9
28018400,2017,2,28,2800308,1,2,1,36,36,71,2,55,18,65,6,39,31,71,2,59,14,56,16,70,3,47,26,15,56,3,68,54,17,45,27,57,10,5,40,30,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,17,4,0,14,3,34,0,0,34,0,0,0,34,383,34,11,3.9
28018435,2017,2,28,2800308,1,2,1,56,79,124,14,101,37,121,20,67,74,136,4,120,20,115,26,135,4,87,52,36,103,14,123,122,19,77,58,105,25,6,64,70,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37,0,0,24,5,0,14,0,41,0,0,41,0,0,0,41,476,41,11,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2017,2,28,2807105,2,2,1,106,186,253,39,198,95,199,95,109,182,287,4,265,25,246,44,286,3,193,97,59,230,15,276,218,75,175,117,247,36,9,176,103,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,0,0,31,3,0,18,3,51,0,0,46,5,0,0,51,611,51,11,3.5
28009487,2017,2,28,2807105,2,2,2,18,23,32,10,31,11,22,19,18,22,39,3,36,6,33,9,41,1,27,15,5,37,2,39,33,8,22,20,35,4,3,29,12,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,7,1,0,6,0,13,0,0,12,1,0,0,13,176,13,13,3.9
28014839,2017,2,28,2807204,2,2,1,31,42,68,6,57,17,55,18,41,31,73,1,71,3,66,8,74,0,56,18,7,64,15,57,66,7,39,33,64,9,2,35,35,...,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,16,0,0,4,0,1,11,1,16,0,1,16,0,0,1,17,198,17,11,3.5
28010132,2017,2,28,2807402,2,2,1,62,76,110,31,92,49,99,42,74,66,132,12,122,21,109,34,138,5,92,51,43,98,27,111,110,31,89,48,101,36,5,70,68,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,13,0,0,17,1,30,0,0,27,3,0,0,30,486,30,16,3.5


In [ ]:
dados_escola.dtypes[dados_escola.dtypes == np.float64]

Ideb-2017    float64
dtype: object

In [ ]:
dataset['ID_DEPENDENCIA_ADM'].unique()

array([2, 1])

In [ ]:
#dataset.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa Ideb/variaveis_derivadas_escola_estudante_5.0.csv')
dataset.to_csv('/content/drive/MyDrive/TCC/variaveis_derivadas_escola_estudante_5.0.csv')

### Discretização dos rótulos, nota do Ideb

**Discretização:** É o processo de transferência de funções contínuas, modelos, variáveis e equações em contrapartes discretas.

In [ ]:
# São três classes média, acima da média e abaixo da média
def discretizacao(ideb, mean_ideb):
    classe = []
    for nota in ideb['Ideb-2017']:
        if (mean_ideb == nota):
            classe.append('média')
        elif (mean_ideb < nota):
            classe.append('acima da média')
        else:
            classe.append('abaixo da média')
    return classe

In [ ]:
#discretizando as notas do Ideb

Ideb = dataset[['Ideb-2017']]

mean_Ideb = round(Ideb.mean(),1)

classe_ideb = discretizacao(Ideb, mean_Ideb['Ideb-2017'])

Ideb.insert(1, 'classe-Ideb', classe_ideb)

Ideb

,Ideb-2017,classe-Ideb
28012631,3.1,abaixo da média
28017846,3.9,acima da média
28017854,2.9,abaixo da média
28018400,3.9,acima da média
28018435,3.7,acima da média
...,...,...
28009398,3.5,acima da média
28009487,3.9,acima da média
28014839,3.5,acima da média
28010132,3.5,acima da média


In [ ]:
#Variavel a ser prevista
y = Ideb['classe-Ideb']

**Seleção de Atributos:**  É um passo da fase de pré-processamento que tem a finalidade de selecionar o melhor subconjunto de atributos que pode substituir o conjunto de dados original. 

https://www.analyticsvidhya.com/blog/2016/12/introduction-to-feature-selection-methods-with-an-example-or-how-to-select-the-right-variables/#:~:text=Embedded%20methods%20combine%20the%20qualities,penalization%20functions%20to%20reduce%20overfitting.

Abordagens:

- Filtro: a relevância dos atributos é avaliada independentemente – não
envolve o uso de um método de AM ;

- Wrapper (Envólucro): Avalia o subconjunto de atributos arti culado ao método de AM que realizará a indução do conceito;

- Embedded (Embutidos): Os métodos incorporados combinam as qualidades dos métodos de filtro e wrapper. Ele é implementado por algoritmos que possuem seus próprios métodos de seleção de recursos integrados.

In [ ]:
#Filter based
# f_classif: ANOVA F-value between label/feature for classification tasks.
#ANOVA stands for Analysis of variance. It is similar to LDA except for the fact that it is operated using one or more categorical 
#independent features and one continuous dependent feature. It provides a statistical test 
#of whether the means of several groups are equal or not.
def selecao_atributos(X, y, quant):
    select = SelectKBest(score_func=f_classif, k=quant)
    #select = SelectKBest(score_func=chi2, k=quant)
    z = select.fit_transform(X, y)
    
    filter = select.get_support(indices=True)
    features = array(X.columns)
    
    return X.iloc[:, filter]

In [ ]:
#Wrapper-based
#from sklearn.cluster import KMeans

def wrapper_based(X, X_transform, y, quant):

  #select_features = RFE(estimator=KMeans(n_clusters=3), n_features_to_select=100)
  select_features = RFE(estimator=SVC(kernel="linear"), n_features_to_select=100)
  '''select_features = RFECV(
      estimator=SVC(kernel="linear"),
      min_features_to_select= quant,
      cv = 5
  )'''
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)
  return X.iloc[:, filter]

In [ ]:
# Embedded-based

def embedded_based(X, X_transform, y, quant):

  select_features = SelectFromModel(LogisticRegression(), max_features=quant)
  #select_features = SelectFromModel(DecisionTreeClassifier(), max_features=quant)
  #select_features = SelectFromModel(SVC(kernel="rbf"), max_features=quant)
  #select_features = SelectFromModel(RandomForestClassifier(), max_features=quant)
  select_features.fit(X_transform, y)

  filter = select_features.get_support(indices=True)

  return X.iloc[:, filter]

**Métricas de Avaliação**


In [ ]:
def acuracia(resultados, classes):
    #resultados = cross_val_predict(modelo, textos, classes, cv=10)  
    return 'Acurácia do modelo: {} %'.format(round(metrics.accuracy_score(classes, resultados)*100, 2))

**Normalização:** é mudar os valores das colunas numéricas no conjunto de dados para usar uma escala comum, sem distorcer as diferenças nos intervalos de valores nem perder informações.

- **Normalização MinMax:** Para cada recurso, o valor mínimo desse recurso é transformado em 0, o valor máximo é transformado em 1 e todos os outros valores são transformados em um decimal entre 0 e 1.

In [ ]:
def minmax(data):
    scaler = MinMaxScaler()
    data_scaler = scaler.fit_transform(data)
    return data_scaler

## Criação dos modelos

**Serão feitas três análises:** Apenas com as variáveis socioeconômicas, com as variáveis de escolas e professores e depois com o dataset completo

Para cada uma serão criados modelos com todos os atributos e também com seleção de atributos, a fim de verificar o de maior acurácia

In [ ]:
def inicializar_modelos():
    naive = MultinomialNB()
    random = RandomForestClassifier(n_estimators=150)
    decision_tree = DecisionTreeClassifier()
    svm = SVC(kernel="linear")
    r_logistic = LogisticRegression(max_iter=250)
    knn = KNeighborsClassifier(n_neighbors=3)
    return naive, random, decision_tree, svm, r_logistic, knn

#### -Primeira análise: variáveis socioeconômicas derivadas da prova SAEB**

In [ ]:
#Foi retirada colunas do 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF'
columns = dataset.columns[3:234]

columns

Index(['ID_MUNICIPIO', 'ID_AREA', 'ID_DEPENDENCIA_ADM', 'ID_LOCALIZACAO',
       'TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q020_A', 'TX_RESP_Q020_B',
       'TX_RESP_Q021_A', 'TX_RESP_Q021_B',
       ...
       'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 'TX_RESP_Q003_E',
       'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I',
       'TX_RESP_Q003_J', 'TX_RESP_Q003_L'],
      dtype='object', length=231)

In [ ]:
X =  dataset[columns]

In [ ]:
#Questões de sexo, raça, ano, mes de nascimento, conformidade com censo
question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H']

#delete_at = ['IN_SITUACAO_CENSO']

X.drop(question, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float]

listOfColumnNames = list(filteredColumns.index)

X[listOfColumnNames]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


""
28012631
28017846
28017854
28018400
28018435
...
28009398
28009487
28014839
28010132


**Divisão entre conjunto de treino e teste**

In [ ]:
X_scaler = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 56.25 %
Naive Bayes: Acurácia do modelo: 68.75 %
Decision Tree: Acurácia do modelo: 62.5 %
SVM: Acurácia do modelo: 81.25 %
Logistic Regression: Acurácia do modelo: 81.25 %
KNN: Acurácia do modelo: 56.25 %


**-Primeira Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [148] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q024_A,TX_RESP_Q025_A,TX_RESP_Q028_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q026_B,TX_RESP_Q032_C,TX_RESP_Q033_A,TX_RESP_Q034_A,TX_RESP_Q035_C,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_A,TX_RESP_Q041_C,TX_RESP_Q042_C,TX_RESP_Q046_A,TX_RESP_Q046_C,TX_RESP_Q047_C,TX_RESP_Q048_B,TX_RESP_Q049_A,TX_RESP_Q049_C,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q056_C,TX_RESP_Q057_C,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_D,TX_RESP_Q045_D,TX_RESP_Q051_B,...,TX_RESP_Q009_C,TX_RESP_Q009_D,TX_RESP_Q009_E,TX_RESP_Q010_B,TX_RESP_Q010_C,TX_RESP_Q010_E,TX_RESP_Q011_B,TX_RESP_Q011_C,TX_RESP_Q011_D,TX_RESP_Q012_A,TX_RESP_Q012_B,TX_RESP_Q012_E,TX_RESP_Q013_A,TX_RESP_Q013_B,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_A,TX_RESP_Q036_C,TX_RESP_Q036_D,TX_RESP_Q036_E,TX_RESP_Q037_C,TX_RESP_Q037_D,TX_RESP_Q016_A,TX_RESP_Q016_F,TX_RESP_Q019_A,TX_RESP_Q019_D,TX_RESP_Q019_E,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_C,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q023_G,TX_RESP_Q003_L
28012631,2,2,1,4,16,15,6,6,1,4,15,10,6,0,7,0,0,3,12,1,1,1,0,0,6,4,0,0,1,4,1,1,0,0,1,1,10,0,1,0,...,1,0,0,2,0,0,13,1,0,15,3,0,11,4,3,0,3,0,0,1,3,0,5,3,8,0,0,1,0,2,1,0,4,4,3,1,2,2,7,1
28017846,1,2,1,4,31,35,21,6,2,6,35,19,9,7,14,3,1,5,18,8,0,10,3,3,12,16,3,0,2,3,1,27,1,3,5,0,13,0,1,3,...,0,0,0,1,0,0,22,1,0,28,13,0,24,14,3,0,10,0,0,2,2,0,5,5,20,0,5,7,0,2,1,9,15,1,4,8,9,0,9,1
28017854,1,2,1,10,77,92,60,9,5,15,80,49,21,15,47,3,1,13,42,21,5,17,4,4,17,49,2,11,0,7,8,48,9,10,16,8,36,1,0,14,...,2,0,0,10,2,0,56,1,0,71,31,0,65,37,1,0,21,4,0,16,4,1,16,13,37,2,17,20,2,18,3,17,32,1,13,19,24,3,24,10
28018400,1,2,1,2,55,65,39,14,3,3,68,33,20,16,33,2,1,8,51,7,0,18,5,3,13,34,1,5,0,5,1,31,1,1,13,0,38,0,1,13,...,2,0,0,7,1,0,51,0,0,41,28,0,26,40,6,1,23,1,0,5,3,1,4,25,19,0,6,7,1,12,0,13,25,8,6,5,19,7,12,7
28018435,1,2,1,14,101,121,67,20,4,14,123,51,43,8,48,10,2,18,73,24,2,19,5,5,9,79,3,13,2,11,1,73,1,3,23,1,56,2,5,19,...,1,0,1,10,0,0,79,1,0,92,42,0,66,59,8,4,29,9,2,6,9,2,12,27,62,4,16,22,3,18,2,22,48,14,13,28,30,6,29,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,2,1,39,198,199,109,25,3,15,276,127,59,33,115,11,2,19,130,66,4,38,5,5,19,160,2,16,2,12,0,161,1,2,34,0,86,4,2,39,...,0,0,0,11,0,0,70,2,0,230,63,0,189,98,5,0,55,6,0,11,13,3,45,57,79,12,34,39,2,46,15,23,43,15,21,57,24,5,46,29
28009487,2,2,2,10,31,22,18,6,1,2,39,20,6,6,9,0,0,1,19,8,0,8,0,0,3,28,0,5,0,2,0,26,0,0,16,0,5,0,0,0,...,0,0,0,1,1,0,7,0,0,35,6,0,32,10,0,0,3,0,0,0,0,0,10,3,13,1,5,6,0,7,4,4,2,1,4,7,2,0,9,4
28014839,2,2,1,6,57,55,41,3,0,15,57,32,19,13,37,2,0,2,30,15,1,12,3,2,6,35,3,9,0,5,2,45,3,4,15,2,36,0,1,2,...,1,0,0,7,0,0,21,0,0,61,14,0,59,13,1,2,15,2,0,2,3,1,14,13,16,6,9,10,0,16,2,14,15,5,10,12,10,2,20,5
28010132,2,2,1,31,92,99,74,21,5,27,111,56,32,16,39,5,2,6,57,31,2,18,4,1,23,66,2,12,0,1,0,63,1,3,34,0,25,1,3,13,...,1,0,0,5,0,0,49,0,0,120,21,0,108,33,1,1,27,2,0,11,6,1,30,29,40,5,18,25,3,24,14,14,11,7,21,22,15,2,34,6


In [ ]:
X_scaler = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 81.25 %
Naive Bayes: Acurácia do modelo: 62.5 %
Decision Tree: Acurácia do modelo: 62.5 %
SVM: Acurácia do modelo: 75.0 %
Logistic Regression: Acurácia do modelo: 81.25 %
KNN: Acurácia do modelo: 75.0 %


**-Primeira Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_scaler, y, 100)

X_embedded

,ID_MUNICIPIO,ID_DEPENDENCIA_ADM,TX_RESP_Q021_B,TX_RESP_Q024_B,TX_RESP_Q027_B,TX_RESP_Q029_A,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q022_B,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_C,TX_RESP_Q033_A,TX_RESP_Q034_B,TX_RESP_Q034_C,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q046_A,TX_RESP_Q046_C,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_C,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q056_B,TX_RESP_Q056_C,TX_RESP_Q057_C,TX_RESP_Q058_B,TX_RESP_Q045_C,TX_RESP_Q051_B
28012631,2800100,2,3,4,0,14,13,6,15,3,9,17,1,7,10,3,6,0,9,1,0,3,6,1,0,0,11,11,0,1,4,0,9,6,1,1,13,0,0,1,1,0
28017846,2800308,2,10,6,2,37,29,12,31,10,30,36,4,18,19,10,9,7,18,8,1,5,15,10,3,3,24,19,3,2,3,0,21,16,1,27,8,1,3,5,4,3
28017854,2800308,2,25,10,2,86,60,40,85,15,55,76,19,52,49,38,21,15,41,10,1,13,37,17,4,4,74,34,2,0,7,8,61,23,8,48,31,9,10,16,1,14
28018400,2800308,2,18,6,2,56,47,26,54,17,45,57,10,30,33,30,20,16,30,8,1,8,14,18,5,3,56,33,1,0,5,1,62,8,1,31,34,1,1,13,2,13
28018435,2800308,2,37,20,4,115,87,52,122,19,77,105,25,70,51,59,43,8,67,21,2,18,41,19,5,5,124,51,3,2,11,1,113,19,1,73,55,1,3,23,5,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2807105,2,95,95,4,246,193,97,218,75,175,247,36,103,127,30,59,33,143,25,2,19,96,38,5,5,255,101,2,2,12,1,248,30,0,161,104,1,2,34,3,39
28009487,2807105,2,11,19,3,33,27,15,33,8,22,35,4,12,20,6,6,6,30,3,0,1,15,8,0,0,39,14,0,0,2,0,37,5,0,26,16,0,0,16,0,0
28014839,2807204,2,17,18,1,66,56,18,66,7,39,64,9,35,32,11,19,13,24,7,0,2,27,12,3,2,63,24,3,0,5,1,51,13,2,45,15,3,4,15,1,2
28010132,2807402,2,49,42,12,109,92,51,110,31,89,101,36,68,56,37,32,16,73,16,2,6,46,18,4,1,106,53,2,0,1,1,88,35,0,63,56,1,3,34,2,13


In [ ]:
X_scaler_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 68.75 %
Naive Bayes: Acurácia do modelo: 37.5 %
Decision Tree: Acurácia do modelo: 31.25 %
SVM: Acurácia do modelo: 18.75 %
Logistic Regression: Acurácia do modelo: 31.25 %
KNN: Acurácia do modelo: 75.0 %


**-Primeira Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_scaler, y, 100)

X_wrapper

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,...,TX_RESP_Q046_A,TX_RESP_Q046_B,TX_RESP_Q046_C,TX_RESP_Q047_A,TX_RESP_Q047_B,TX_RESP_Q047_C,TX_RESP_Q048_A,TX_RESP_Q048_B,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_A,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_A,TX_RESP_Q055_B,TX_RESP_Q055_C,TX_RESP_Q056_A,TX_RESP_Q056_B,TX_RESP_Q056_C,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q045_D,TX_RESP_Q051_A,TX_RESP_Q051_B
28012631,2800100,2,2,1,15,4,16,3,15,4,6,13,19,0,13,6,14,5,18,1,13,6,1,18,4,15,15,3,9,9,17,1,1,12,7,0,6,10,3,2,...,1,15,0,3,12,0,11,6,0,4,11,0,17,0,1,14,4,0,9,6,1,1,13,0,3,12,0,15,1,1,10,5,3,0,11,6,1,1,18,0
28017846,2800308,1,2,1,37,4,31,10,35,6,21,18,39,2,35,6,37,4,39,2,29,12,11,30,6,35,31,10,30,10,36,4,0,22,18,1,12,19,10,5,...,10,24,3,14,20,3,24,12,3,16,19,3,38,0,2,36,3,0,21,16,1,27,8,1,7,27,3,33,5,0,13,25,3,0,24,12,4,1,36,3
28017854,2800308,1,2,1,93,10,77,25,92,10,60,41,100,2,92,9,86,14,95,5,60,40,37,65,15,80,85,15,55,44,76,19,8,48,52,4,17,49,38,17,...,17,65,4,18,64,4,74,17,3,49,34,2,87,11,0,79,7,8,61,23,8,48,31,9,12,64,10,75,16,8,36,53,13,1,69,31,1,0,88,14
28018400,2800308,1,2,1,71,2,55,18,65,6,39,31,71,2,59,14,56,16,70,3,47,26,15,56,3,68,54,17,45,27,57,10,5,40,30,3,10,33,30,12,...,18,47,5,21,46,3,56,13,1,34,33,1,64,5,0,63,5,1,62,8,1,31,34,1,8,58,1,56,13,0,38,31,4,0,44,25,2,1,60,13
28018435,2800308,1,2,1,124,14,101,37,121,20,67,74,136,4,120,20,115,26,135,4,87,52,36,103,14,123,122,19,77,58,105,25,6,64,70,6,31,51,59,19,...,19,107,5,35,92,5,124,9,4,79,51,3,119,13,2,123,11,1,113,19,1,73,55,1,10,112,3,107,23,1,56,70,13,2,71,59,5,5,120,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2807105,2,2,1,253,39,198,95,199,95,109,182,287,4,265,25,246,44,286,3,193,97,59,230,15,276,218,75,175,117,247,36,9,176,103,15,137,127,30,47,...,38,223,5,72,187,5,255,19,2,160,101,2,263,16,2,256,12,1,248,30,0,161,104,1,34,227,2,242,34,0,86,151,53,4,234,55,3,2,245,39
28009487,2807105,2,2,2,32,10,31,11,22,19,18,22,39,3,36,6,33,9,41,1,27,15,5,37,2,39,33,8,22,20,35,4,3,29,12,1,16,20,6,7,...,8,34,0,14,28,0,39,3,0,28,14,0,37,5,0,40,2,0,37,5,0,26,16,0,5,36,0,25,16,0,5,31,5,0,37,5,0,0,42,0
28014839,2807204,2,2,1,68,6,57,17,55,18,41,31,73,1,71,3,66,8,74,0,56,18,7,64,15,57,66,7,39,33,64,9,2,35,35,4,30,32,11,11,...,12,47,3,24,35,2,63,6,0,35,24,3,57,9,0,62,5,1,51,13,2,45,15,3,11,48,4,49,15,2,36,33,3,0,52,18,1,1,69,2
28010132,2807402,2,2,1,110,31,92,49,99,42,74,66,132,12,122,21,109,34,138,5,92,51,43,98,27,111,110,31,89,48,101,36,5,70,68,6,51,56,37,21,...,18,99,4,30,89,1,106,23,0,66,53,2,121,12,0,132,1,1,88,35,0,63,56,1,10,108,3,91,34,0,25,74,41,1,91,46,2,3,122,13


In [ ]:
X_scaler_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_scaler_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 68.75 %
Naive Bayes: Acurácia do modelo: 62.5 %
Decision Tree: Acurácia do modelo: 68.75 %
SVM: Acurácia do modelo: 62.5 %
Logistic Regression: Acurácia do modelo: 62.5 %
KNN: Acurácia do modelo: 75.0 %


#### -Segunda análise: Análise com o dataset completo

In [ ]:
#Lembrar de tratar datas

'''question = ['TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D', 'CO_REGIAO',
            'TP_SITUACAO_FUNCIONAMENTO', 'TP_REGULAMENTACAO',
            'CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
            'TX_RESP_Q004_E', 'TX_RESP_Q004_F', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H', 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 
            'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'IN_SITUACAO_CENSO', 'NU_ANO_CENSO', 'CO_UF', 'CO_MUNICIPIO', 'Ideb-2017']'''

'''delete = ['CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_DISTRITO', 'TP_LOCALIZACAO',  'CO_ORGAO_REGIONAL', 'TP_DEPENDENCIA',
          'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'ID_MUNICIPIO', 'IN_SITUACAO_CENSO', 'Ideb-2017']'''


#X = dataset.drop(delete, axis=1)

'''X = dataset.drop(['Ideb-2017','IN_SITUACAO_CENSO', 'TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q002_A', 'TX_RESP_Q002_B', 'TX_RESP_Q002_C', 'TX_RESP_Q002_D', 
            'TX_RESP_Q002_E', 'TX_RESP_Q002_F', 'TX_RESP_Q003_A', 'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 
            'TX_RESP_Q003_E', 'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I', 'TX_RESP_Q003_J', 
            'TX_RESP_Q003_L', 'TX_RESP_Q004_A', 'TX_RESP_Q004_B', 'TX_RESP_Q004_C', 'TX_RESP_Q004_D',], axis=1)'''

X = dataset.drop(['Ideb-2017'], axis=1)

In [ ]:
X

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,TX_RESP_Q024_A,TX_RESP_Q024_B,TX_RESP_Q025_A,TX_RESP_Q025_B,TX_RESP_Q027_A,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q029_B,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q031_A,TX_RESP_Q031_B,TX_RESP_Q038_A,TX_RESP_Q038_B,TX_RESP_Q043_A,TX_RESP_Q043_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q018_B,TX_RESP_Q018_C,TX_RESP_Q022_A,TX_RESP_Q022_B,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf
28012631,2017,2,28,2800100,2,2,1,12,6,15,4,16,3,15,4,6,13,19,0,13,6,14,5,18,1,13,6,1,18,4,15,15,3,9,9,17,1,1,12,7,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,8,0,0,4,1,0,3,0,8,0,0,7,1,0,0,8,22,8,2
28017846,2017,2,28,2800308,1,2,1,21,20,37,4,31,10,35,6,21,18,39,2,35,6,37,4,39,2,29,12,11,30,6,35,31,10,30,10,36,4,0,22,18,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,10,2,0,10,0,22,0,0,22,0,0,0,22,121,22,5
28017854,2017,2,28,2800308,1,2,1,39,62,93,10,77,25,92,10,60,41,100,2,92,9,86,14,95,5,60,40,37,65,15,80,85,15,55,44,76,19,8,48,52,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,0,0,17,2,0,13,0,32,0,0,32,0,0,0,32,301,32,9
28018400,2017,2,28,2800308,1,2,1,36,36,71,2,55,18,65,6,39,31,71,2,59,14,56,16,70,3,47,26,15,56,3,68,54,17,45,27,57,10,5,40,30,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,17,4,0,14,3,34,0,0,34,0,0,0,34,383,34,11
28018435,2017,2,28,2800308,1,2,1,56,79,124,14,101,37,121,20,67,74,136,4,120,20,115,26,135,4,87,52,36,103,14,123,122,19,77,58,105,25,6,64,70,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37,0,0,24,5,0,14,0,41,0,0,41,0,0,0,41,476,41,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2017,2,28,2807105,2,2,1,106,186,253,39,198,95,199,95,109,182,287,4,265,25,246,44,286,3,193,97,59,230,15,276,218,75,175,117,247,36,9,176,103,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,0,0,31,3,0,18,3,51,0,0,46,5,0,0,51,611,51,11
28009487,2017,2,28,2807105,2,2,2,18,23,32,10,31,11,22,19,18,22,39,3,36,6,33,9,41,1,27,15,5,37,2,39,33,8,22,20,35,4,3,29,12,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,7,1,0,6,0,13,0,0,12,1,0,0,13,176,13,13
28014839,2017,2,28,2807204,2,2,1,31,42,68,6,57,17,55,18,41,31,73,1,71,3,66,8,74,0,56,18,7,64,15,57,66,7,39,33,64,9,2,35,35,...,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,16,0,0,4,0,1,11,1,16,0,1,16,0,0,1,17,198,17,11
28010132,2017,2,28,2807402,2,2,1,62,76,110,31,92,49,99,42,74,66,132,12,122,21,109,34,138,5,92,51,43,98,27,111,110,31,89,48,101,36,5,70,68,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,13,0,0,17,1,30,0,0,27,3,0,0,30,486,30,16


In [ ]:
filteredColumns = X.dtypes[X.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

#X[listOfColumnNames].fillna(-1, inplace=True)

#X[listOfColumnNames] = X[listOfColumnNames].astype(int)

X[listOfColumnNames].isnull().sum()

Series([], dtype: float64)

In [ ]:
X_transform = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 56.25 %
Naive Bayes: Acurácia do modelo: 75.0 %
Decision Tree: Acurácia do modelo: 56.25 %
SVM: Acurácia do modelo: 50.0 %
Logistic Regression: Acurácia do modelo: 56.25 %
KNN: Acurácia do modelo: 68.75 %


**-Segunda Análise: Aplicando Seleção de Atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,ID_MUNICIPIO,ID_AREA,TX_RESP_Q028_B,TX_RESP_Q043_A,TX_RESP_Q033_A,TX_RESP_Q040_B,TX_RESP_Q042_C,TX_RESP_Q050_B,TX_RESP_Q054_B,TX_RESP_Q055_C,TX_RESP_Q039_D,TX_RESP_Q051_D,TX_RESP_Q053_B,TX_RESP_Q005_A,TX_RESP_Q005_E,TX_RESP_Q006_E,TX_RESP_Q007_C,TX_RESP_Q009_C,TX_RESP_Q009_E,TX_RESP_Q010_C,TX_RESP_Q010_E,TX_RESP_Q012_B,TX_RESP_Q012_E,TX_RESP_Q013_C,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_A,TX_RESP_Q036_C,TX_RESP_Q036_E,TX_RESP_Q037_C,TX_RESP_Q002_B,TX_RESP_Q002_E,TX_RESP_Q002_F,TX_RESP_Q016_A,TX_RESP_Q019_F,TX_RESP_Q023_F,...,IN_SALA_PROFESSOR,IN_LABORATORIO_INFORMATICA,IN_QUADRA_ESPORTES_COBERTA,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BIBLIOTECA_SALA_LEITURA,IN_DEPENDENCIAS_PNE,IN_BANHEIRO_CHUVEIRO,IN_REFEITORIO,IN_ALMOXARIFADO,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,IN_EQUIP_TV,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_DVD,IN_EQUIP_COPIADORA,IN_EQUIP_IMPRESSORA,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_SOM,IN_EQUIP_FAX,NU_EQUIP_TV,NU_EQUIP_DVD,NU_EQUIP_PARABOLICA,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_FUNCIONARIOS,IN_FUNDAMENTAL_CICLOS,IN_FINAL_SEMANA,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_FUND_AF,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF
28012631,2800100,2,6,4,0,0,1,0,4,1,0,0,8,1,0,0,2,1,0,0,0,3,0,3,3,0,0,1,3,0,5,3,0,0,7,0,1,0,4,2,...,1,1,0,0,0,0,1,1,0,1,0,1,0,1,1,0,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,17,0,0,1,0,0,0,1,0
28017846,2800308,1,6,6,7,1,0,0,3,1,0,2,10,0,0,0,0,0,0,0,0,13,0,3,10,0,0,2,2,0,5,5,0,5,10,3,3,0,1,0,...,1,0,0,0,0,1,1,1,0,0,1,1,1,1,1,0,0,1,0,1,0,1,1,1,0,3,1,0,2,1,0,55,0,0,0,0,1,0,0,0
28017854,2800308,1,9,15,15,1,5,11,7,8,1,0,41,5,1,0,6,2,0,2,0,31,0,1,21,4,0,16,4,1,16,13,2,17,24,2,6,2,1,3,...,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,2,2,0,0,1,0,81,0,1,0,0,1,0,0,0
28018400,2800308,1,14,3,16,1,0,5,5,1,0,0,32,4,1,0,2,2,0,1,0,28,0,6,23,1,0,5,3,1,4,25,0,6,22,2,2,1,8,7,...,1,1,1,0,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,3,1,0,1,2,1,75,0,0,0,0,1,0,0,0
28018435,2800308,1,20,14,8,2,2,13,11,1,2,2,63,2,2,1,2,1,1,0,0,42,0,8,29,9,2,6,9,2,12,27,4,16,20,2,6,3,14,6,...,1,1,1,0,1,1,0,1,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,1,0,4,1,0,0,6,0,86,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2807105,2,25,15,33,2,4,16,12,0,4,2,51,10,3,0,5,0,0,0,0,63,0,5,55,6,0,11,13,3,45,57,12,34,34,2,8,2,15,5,...,1,1,1,0,1,1,0,1,1,1,1,0,0,1,0,1,0,1,0,1,1,1,0,1,0,5,5,0,0,5,0,90,0,1,0,0,0,0,0,0
28009487,2807105,2,6,2,6,0,0,5,2,0,0,0,15,0,0,0,0,0,0,1,0,6,0,0,3,0,0,0,0,0,10,3,1,5,1,0,2,0,1,0,...,0,1,1,0,1,1,1,1,1,1,0,1,0,1,1,0,0,1,0,1,0,1,1,1,0,2,1,0,2,2,0,35,0,1,0,0,1,0,0,0
28014839,2807204,2,3,15,13,0,1,9,5,2,0,1,22,2,0,0,2,1,0,0,0,14,0,1,15,2,0,2,3,1,14,13,6,9,15,1,6,0,5,2,...,1,1,1,0,1,1,0,1,0,0,0,1,0,1,1,0,0,1,0,0,1,1,1,1,0,7,0,0,3,0,0,55,1,0,1,0,1,1,1,0
28010132,2807402,2,21,27,16,2,2,12,1,0,1,2,44,6,1,0,5,1,0,0,0,21,0,1,27,2,0,11,6,1,30,29,5,18,18,0,7,3,7,2,...,1,0,1,0,1,1,0,1,1,0,0,1,0,1,1,0,0,1,0,1,1,1,1,1,0,3,1,1,2,4,0,53,0,0,0,0,1,0,0,0


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 87.5 %
Naive Bayes: Acurácia do modelo: 81.25 %
Decision Tree: Acurácia do modelo: 62.5 %
SVM: Acurácia do modelo: 81.25 %
Logistic Regression: Acurácia do modelo: 75.0 %
KNN: Acurácia do modelo: 43.75 %


**-Segunda Análise: Aplicando Seleção de Atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_MUNICIPIO,ID_AREA,TX_RESP_Q028_B,TX_RESP_Q029_B,TX_RESP_Q043_A,TX_RESP_Q033_A,TX_RESP_Q040_B,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q050_B,TX_RESP_Q039_D,TX_RESP_Q051_D,TX_RESP_Q053_B,TX_RESP_Q006_A,TX_RESP_Q006_D,TX_RESP_Q006_E,TX_RESP_Q007_C,TX_RESP_Q009_E,TX_RESP_Q010_C,TX_RESP_Q010_E,TX_RESP_Q012_B,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q013_C,TX_RESP_Q013_D,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_E,TX_RESP_Q037_C,TX_RESP_Q002_B,TX_RESP_Q002_E,TX_RESP_Q016_A,TX_RESP_Q023_F,TX_RESP_Q004_B,TX_RESP_Q004_G,TX_RESP_Q003_B,...,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BIBLIOTECA_SALA_LEITURA,IN_DEPENDENCIAS_PNE,IN_BANHEIRO_CHUVEIRO,IN_ALMOXARIFADO,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,NU_SALAS_UTILIZADAS,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_COPIADORA,IN_EQUIP_RETROPROJETOR,IN_EQUIP_IMPRESSORA,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_SOM,IN_EQUIP_FAX,NU_EQUIP_TV,NU_EQUIP_VIDEOCASSETE,NU_EQUIP_DVD,NU_EQUIP_PARABOLICA,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_COMP_ALUNO,IN_BANDA_LARGA,NU_FUNCIONARIOS,IN_FUNDAMENTAL_CICLOS,IN_MATERIAL_ESP_QUILOMBOLA,IN_MATERIAL_ESP_NAO_UTILIZA,IN_EJA,IN_PROFISSIONALIZANTE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_EJA_FUND,IN_COMUM_EJA_PROF
28012631,2800100,2,6,5,4,0,0,0,1,0,0,0,8,10,0,0,2,0,0,0,3,0,0,3,0,3,0,0,1,3,0,0,0,7,0,0,2,0,0,2,...,0,0,0,1,1,0,1,1,0,1,1,0,0,4,0,0,1,0,1,1,0,1,0,0,0,1,1,0,10,1,17,0,0,1,1,0,0,0,0,0
28017846,2800308,1,6,4,6,7,1,5,0,0,0,2,10,17,0,0,0,0,0,0,13,0,0,3,0,10,0,0,2,2,0,0,5,10,3,0,0,1,0,4,...,0,0,1,1,1,0,0,1,1,1,1,0,0,14,0,0,0,1,1,1,0,3,0,1,0,2,1,0,1,1,55,0,0,1,0,0,0,1,0,0
28017854,2800308,1,9,14,15,15,1,11,5,11,1,0,41,47,1,0,6,0,2,0,31,0,0,1,0,21,4,0,16,4,1,2,17,24,2,2,3,2,5,7,...,0,1,1,0,1,1,1,1,1,1,1,0,0,20,0,1,1,1,0,1,0,2,0,2,0,0,1,0,22,1,81,0,0,1,0,0,0,1,0,0
28018400,2800308,1,14,16,3,16,1,2,0,5,0,0,32,34,0,0,2,0,1,0,28,0,0,6,1,23,1,0,5,3,1,0,6,22,2,1,7,5,0,6,...,0,1,1,1,1,1,0,1,1,1,0,0,0,17,1,1,1,1,1,1,1,3,1,1,0,1,2,1,25,1,75,0,0,1,0,0,0,1,0,0
28018435,2800308,1,20,26,14,8,2,14,2,13,2,2,63,70,0,1,2,1,0,0,42,0,0,8,4,29,9,2,6,9,2,4,16,20,2,3,6,6,6,13,...,0,1,1,0,1,0,1,1,0,1,0,0,0,12,0,1,1,1,0,1,0,4,0,1,0,0,6,0,26,1,86,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2807105,2,25,44,15,33,2,30,4,16,4,2,51,77,0,0,5,0,0,0,63,0,0,5,0,55,6,0,11,13,3,12,34,34,2,2,5,6,15,28,...,0,1,1,0,1,1,1,0,0,1,0,1,0,14,0,1,1,1,0,1,0,5,0,5,0,0,5,0,24,1,90,0,0,1,0,0,0,0,0,0
28009487,2807105,2,6,9,2,6,0,3,0,5,0,0,15,11,0,0,0,0,1,0,6,0,0,0,0,3,0,0,0,0,0,1,5,1,0,0,0,0,4,3,...,0,1,1,1,1,1,1,1,0,1,1,0,0,6,0,0,1,1,1,1,0,2,0,1,0,2,2,0,8,1,35,0,0,1,0,0,0,1,0,0
28014839,2807204,2,3,8,15,13,0,7,1,9,0,1,22,22,0,0,2,0,0,0,14,0,0,1,2,15,2,0,2,3,1,6,9,15,1,0,2,3,4,8,...,0,1,1,0,1,0,0,1,0,1,1,0,0,10,0,1,0,1,1,1,0,7,0,0,0,3,0,0,16,1,55,1,0,1,1,0,1,1,1,0
28010132,2807402,2,21,34,27,16,2,20,2,12,1,2,44,36,1,0,5,0,0,0,21,0,0,1,1,27,2,0,11,6,1,5,18,18,0,3,2,3,10,5,...,0,1,1,0,1,1,0,1,0,1,1,0,0,11,0,1,0,1,1,1,0,3,0,1,1,2,4,0,2,1,53,0,0,1,0,0,0,1,0,0


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 87.5 %
Naive Bayes: Acurácia do modelo: 68.75 %
Decision Tree: Acurácia do modelo: 75.0 %
SVM: Acurácia do modelo: 93.75 %
Logistic Regression: Acurácia do modelo: 75.0 %
KNN: Acurácia do modelo: 62.5 %


**-Segunda Análise: Aplicando Seleção de Atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X, y, 100)

X_selection

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   1   2 153 234 236 239 242 250 251 252 253 254 255 256 261 264 265
 266 267 268 269 275 276 277 278 287 291 292 294 295 306 310 325 341 344
 350 352 353 355 356 357 358 359 360 363 364 367 369 370 371 372 373 374
 375 376 380 381 382 384 386 387 394 398] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_AREA,ID_LOCALIZACAO,TX_RESP_Q020_B,TX_RESP_Q025_A,TX_RESP_Q028_B,TX_RESP_Q030_B,TX_RESP_Q043_A,TX_RESP_Q033_A,TX_RESP_Q042_C,TX_RESP_Q046_C,TX_RESP_Q049_C,TX_RESP_Q057_C,TX_RESP_Q039_D,TX_RESP_Q053_B,TX_RESP_Q060_B,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q005_C,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q006_E,TX_RESP_Q007_C,TX_RESP_Q009_C,TX_RESP_Q009_E,TX_RESP_Q010_E,TX_RESP_Q011_C,TX_RESP_Q012_B,TX_RESP_Q012_E,TX_RESP_Q013_B,TX_RESP_Q013_C,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_A,TX_RESP_Q036_E,TX_RESP_Q037_D,TX_RESP_Q002_A,...,TP_LOCALIZACAO,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_PREDIO_COMPARTILHADO,IN_ESGOTO_REDE_PUBLICA,IN_SALA_DIRETORIA,IN_SALA_PROFESSOR,IN_LABORATORIO_INFORMATICA,IN_QUADRA_ESPORTES_COBERTA,IN_QUADRA_ESPORTES_DESCOBERTA,IN_QUADRA_ESPORTES,IN_BIBLIOTECA,IN_SALA_LEITURA,IN_BIBLIOTECA_SALA_LEITURA,IN_BANHEIRO_PNE,IN_BANHEIRO_CHUVEIRO,IN_ALMOXARIFADO,NU_SALAS_EXISTENTES,IN_EQUIP_TV,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_FAX,IN_EQUIP_FOTO,NU_EQUIP_TV,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_SOM,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_EQUIP_FOTO,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_COMP_ALUNO,NU_FUNCIONARIOS,IN_FINAL_SEMANA,IN_EJA,IN_COMUM_FUND_AF,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesEfetivos,QtdeTurmasProf
28012631,2,1,4,6,6,1,4,0,1,0,0,0,0,8,9,0,1,1,1,0,0,2,1,0,0,1,3,0,4,3,3,0,0,1,3,0,5,0,1,4,...,1,1,1,0,0,1,1,1,1,0,0,0,0,1,1,0,1,1,4,1,1,0,0,1,1,1,1,0,0,12,2,10,17,0,1,0,1,0,7,8
28017846,1,1,4,21,6,2,6,7,0,3,3,3,0,10,22,0,0,12,3,0,0,0,0,0,0,1,13,0,14,3,10,0,0,2,2,0,5,0,7,2,...,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,0,0,1,16,1,1,0,1,3,2,1,1,0,1,5,5,1,55,0,0,1,2,0,22,22
28017854,1,1,10,60,9,5,15,15,5,4,2,10,1,41,49,1,5,30,7,1,0,6,2,0,0,1,31,0,37,1,21,4,0,16,4,1,16,2,20,14,...,1,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,1,20,1,0,0,1,2,0,2,1,0,3,25,3,22,81,1,0,1,2,0,32,32
28018400,1,1,2,39,14,3,3,16,0,5,1,1,0,32,31,1,4,31,8,1,0,2,2,0,0,0,28,0,40,6,23,1,0,5,3,1,4,0,7,8,...,1,1,1,0,0,1,1,1,1,1,0,1,1,1,1,1,0,1,17,1,1,1,1,3,1,2,2,1,1,30,5,25,75,0,0,1,4,0,34,34
28018435,1,1,14,67,20,4,14,8,2,5,3,3,2,63,73,0,2,33,5,2,1,2,1,1,0,1,42,0,59,8,29,9,2,6,9,2,12,4,22,14,...,1,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,1,13,1,0,0,1,4,0,3,6,0,5,34,8,26,86,1,0,1,5,0,41,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,1,39,109,25,3,15,33,4,5,2,2,4,51,144,0,10,36,8,3,0,5,0,0,0,2,63,0,98,5,55,6,0,11,13,3,45,12,39,40,...,1,1,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,0,14,1,0,0,1,5,0,7,5,0,4,33,4,24,90,1,0,0,3,0,46,51
28009487,2,2,10,18,6,1,2,6,0,0,0,0,0,15,29,0,0,5,1,0,0,0,0,0,0,0,6,0,10,0,3,0,0,0,0,0,10,1,6,4,...,2,1,1,0,0,1,1,0,1,1,0,1,1,1,1,1,1,1,6,1,1,0,1,2,2,1,2,0,1,12,4,8,35,1,0,1,1,0,12,13
28014839,2,1,6,41,3,0,15,13,1,3,3,4,0,22,34,0,2,12,0,0,0,2,1,0,0,0,14,0,13,1,15,2,0,2,3,1,14,6,10,4,...,1,1,1,0,0,0,1,1,1,1,0,1,1,0,1,1,0,1,11,1,1,0,0,7,3,24,0,0,0,56,3,16,55,0,1,1,0,1,16,17
28010132,2,1,31,74,21,5,27,16,2,4,2,3,1,44,33,31,6,21,1,1,0,5,1,0,0,0,21,0,33,1,27,2,0,11,6,1,30,5,25,23,...,1,1,1,0,0,1,0,1,0,1,0,1,1,0,1,1,0,1,11,1,1,0,1,3,2,2,4,0,1,8,2,2,53,0,0,1,0,0,27,30


In [ ]:
X_transform = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 96.05 %
Naive Bayes: Acurácia do modelo: 78.95 %
Decision Tree: Acurácia do modelo: 92.11 %
SVM: Acurácia do modelo: 93.42 %
Logistic Regression: Acurácia do modelo: 94.74 %
KNN: Acurácia do modelo: 69.74 %


**Segunda Análise: Unindo subconjunto de melhores atributos**

In [ ]:
X_all = pd.concat([X_selection, X_wrapper], axis=1)

X_all = pd.concat([X_all, X_embedded], axis=1)

In [ ]:
#Removendo colunas duplicadas
X_new_all = X_all.T.drop_duplicates().T

In [ ]:
X_transform = minmax(X_new_all)
#X_transform = minmax(X_all)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 94.74 %
Naive Bayes: Acurácia do modelo: 81.58 %
Decision Tree: Acurácia do modelo: 90.79 %
SVM: Acurácia do modelo: 94.74 %
Logistic Regression: Acurácia do modelo: 96.05 %
KNN: Acurácia do modelo: 73.68 %


Resultado com Todos atributos

- Random forest: Acurácia do modelo: 96.05 %
- Naive Bayes: Acurácia do modelo: 82.89 %
- Decision Tree: Acurácia do modelo: 93.42 %
- SVM: Acurácia do modelo: 97.37 %
- Logistic Regression: Acurácia do modelo: 97.37 %
- KNN: Acurácia do modelo: 76.32 %

**Segunda Análise: Aplicando a técnica merge**

Link da Dissertação: https://www.dcc.ufmg.br/pos/cursos/defesas/1930M.PDF


In [ ]:
#Metodo Merge combina os atributos mais frequentes nos melhores conjuntos de seleção 
#Cada atributo irá apresentar um score de acordo com sua presença em um subconjunto de dados
def metodo_merge(selection, embedded, wrapper, ranking):
  for atributo in selection:
    score = ranking[atributo]
    if (atributo in embedded):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in embedded:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in wrapper):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score

  for atributo in wrapper:
    score = ranking[atributo]
    if (atributo in selection):
      score += 1
      embedded.remove(atributo)
    if (atributo in embedded):
      score +=1
      wrapper.remove(atributo)
    ranking[atributo] = score
 
  return ranking      

In [ ]:
#Gerando Listas com os melhores atributos de cada abordagem
selection = list(X_selection.columns)
embedded = list(X_embedded.columns)
wrapper = list(X_wrapper.columns)

#Criando dicionário
values = [1]*300

all = selection + embedded + wrapper

ranking = dict(zip(all, values))

ranking = metodo_merge(selection, embedded, wrapper, ranking)

#Filtrando os atributos que estão em mais de um subconjunto

dic_aux = ranking.items()

filtered = {key:value for key, value in dic_aux if value > 1}

#Gerando X após o merge

X_merge = dataset[list(filtered.keys())]

In [ ]:
X_transform = minmax(X_merge)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_transform)
naive_result = naive.predict(X_transform)
decision_result = decision_tree.predict(X_transform)
svm_result = svm.predict(X_transform)
logistic_result = r_logistic.predict(X_transform)
knn_result = knn.predict(X_transform)

print("Random forest: " + acuracia(random_result, y))
print("Naive Bayes: " + acuracia(naive_result, y))
print("Decision Tree: " + acuracia(decision_result, y))
print("SVM: " + acuracia(svm_result, y))
print("Logistic Regression: "+acuracia(logistic_result, y))
print("KNN: "+acuracia(knn_result, y))

Random forest: Acurácia do modelo: 96.05 %
Naive Bayes: Acurácia do modelo: 82.89 %
Decision Tree: Acurácia do modelo: 93.42 %
SVM: Acurácia do modelo: 93.42 %
Logistic Regression: Acurácia do modelo: 93.42 %
KNN: Acurácia do modelo: 76.32 %


In [ ]:
dados_para_exportar = pd.DataFrame(columns=X_merge.columns, data=X_merge.values, index=X_merge.index)
dados_para_exportar.insert(len(dados_para_exportar.columns), "Valor Real Ideb", y.to_numpy())
dados_para_exportar.insert(len(dados_para_exportar.columns), "Previsão Ideb", random_result)


dados_para_exportar

,ID_AREA,TX_RESP_Q028_B,TX_RESP_Q043_A,TX_RESP_Q033_A,TX_RESP_Q042_C,TX_RESP_Q039_D,TX_RESP_Q053_B,TX_RESP_Q005_A,TX_RESP_Q005_E,TX_RESP_Q006_E,TX_RESP_Q007_C,TX_RESP_Q009_C,TX_RESP_Q009_E,TX_RESP_Q010_E,TX_RESP_Q012_B,TX_RESP_Q012_E,TX_RESP_Q013_C,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_A,TX_RESP_Q036_E,TX_RESP_Q002_B,TX_RESP_Q002_E,TX_RESP_Q016_A,TX_RESP_Q019_F,TX_RESP_Q023_F,TX_RESP_Q004_B,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_I,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_MESORREGIAO,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,...,IN_EQUIP_TV,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_FAX,NU_EQUIP_TV,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_COMP_ALUNO,NU_FUNCIONARIOS,IN_FINAL_SEMANA,IN_EJA,IN_COMUM_FUND_AF,ID_MUNICIPIO,TX_RESP_Q040_B,TX_RESP_Q050_B,TX_RESP_Q051_D,TX_RESP_Q010_C,TX_RESP_Q037_C,TX_RESP_Q003_D,CO_MUNICIPIO,CO_DISTRITO,IN_ESGOTO_FOSSA,IN_DEPENDENCIAS_PNE,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_COPIADORA,IN_EQUIP_IMPRESSORA,IN_EQUIP_SOM,NU_EQUIP_DVD,NU_EQUIP_PARABOLICA,IN_FUNDAMENTAL_CICLOS,IN_PROFISSIONALIZANTE,IN_COMUM_EJA_FUND,IN_COMUM_EJA_PROF,Valor Real Ideb,Previsão Ideb
28012631,2,6,4,0,1,0,8,1,0,0,2,1,0,0,3,0,3,3,0,0,1,3,0,5,0,7,0,0,4,2,0,0,0,3,6,1486944000000000000,1513900800000000000,2803,1,1,...,1,1,0,1,1,1,0,10,17,0,1,0,2800100,0,0,0,0,0,1,2800100,280010005,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,abaixo da média,abaixo da média
28017846,1,6,6,7,0,0,10,0,0,0,0,0,0,0,13,0,3,10,0,0,2,2,0,5,0,10,3,0,1,0,1,0,0,3,0,1494201600000000000,1519776000000000000,2803,1,1,...,1,1,0,3,2,1,0,1,55,0,0,1,2800308,1,0,2,0,5,2,2800308,280030805,0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,acima da média,acima da média
28017854,1,9,15,15,5,1,41,5,1,0,6,2,0,0,31,0,1,21,4,0,16,4,1,16,2,24,2,2,1,3,2,5,6,11,0,1491782400000000000,1519344000000000000,2803,1,1,...,1,0,0,2,0,1,0,22,81,1,0,1,2800308,1,11,0,2,17,4,2800308,280030805,0,1,1,1,1,0,0,0,1,1,1,2,0,0,0,0,0,abaixo da média,abaixo da média
28018400,1,14,3,16,0,0,32,4,1,0,2,2,0,0,28,0,6,23,1,0,5,3,1,4,0,22,2,1,8,7,5,0,0,3,0,1492387200000000000,1519862400000000000,2803,1,1,...,1,1,1,3,1,2,1,25,75,0,0,1,2800308,1,5,0,1,6,8,2800308,280030805,0,1,1,1,0,0,0,1,1,1,1,1,0,0,0,0,0,acima da média,acima da média
28018435,1,20,14,8,2,2,63,2,2,1,2,1,1,0,42,0,8,29,9,2,6,9,2,12,4,20,2,3,14,6,6,6,5,11,0,1489363200000000000,1515196800000000000,2803,1,1,...,1,0,0,4,0,6,0,26,86,1,0,1,2800308,2,13,2,0,16,14,2800308,280030805,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,acima da média,acima da média
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,25,15,33,4,4,51,10,3,0,5,0,0,0,63,0,5,55,6,0,11,13,3,45,12,34,2,2,15,5,6,15,16,18,2,1493683200000000000,1518739200000000000,2802,1,1,...,1,0,0,5,0,5,0,24,90,1,0,0,2807105,2,16,2,0,34,24,2807105,280710505,1,1,0,1,0,1,0,0,1,1,1,5,0,0,0,0,0,acima da média,acima da média
28009487,2,6,2,6,0,0,15,0,0,0,0,0,0,0,6,0,0,3,0,0,0,0,0,10,1,1,0,0,1,0,0,4,1,1,2,1490745600000000000,1516838400000000000,2802,1,1,...,1,1,0,2,2,2,0,8,35,1,0,1,2807105,0,5,0,1,5,4,2807105,280710505,0,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,acima da média,acima da média
28014839,2,3,15,13,1,0,22,2,0,0,2,1,0,0,14,0,1,15,2,0,2,3,1,14,6,15,1,0,5,2,3,4,4,5,5,1488412800000000000,1517011200000000000,2803,1,1,...,1,1,0,7,3,0,0,16,55,0,1,1,2807204,0,9,1,0,9,5,2807204,280720405,1,0,0,1,1,0,0,0,1,1,1,0,0,1,0,1,0,acima da média,acima da média
28010132,2,21,27,16,2,1,44,6,1,0,5,1,0,0,21,0,1,27,2,0,11,6,1,30,5,18,0,3,7,2,3,10,12,12,2,1488758400000000000,1516320000000000000,2802,1,1,...,1,1,0,3,2,4,0,2,53,0,0,1,2807402,2,12,2,0,18,13,2807402,280740205,1,1,0,1,1,0,0,0,1,1,1,1,1,0,0,0,0,a

In [ ]:
dados_para_exportar = dados_para_exportar.loc[dados_para_exportar['Valor Real Ideb'] ==  dados_para_exportar['Previsão Ideb']]


dados_para_exportar

,ID_AREA,TX_RESP_Q028_B,TX_RESP_Q043_A,TX_RESP_Q033_A,TX_RESP_Q042_C,TX_RESP_Q039_D,TX_RESP_Q053_B,TX_RESP_Q005_A,TX_RESP_Q005_E,TX_RESP_Q006_E,TX_RESP_Q007_C,TX_RESP_Q009_C,TX_RESP_Q009_E,TX_RESP_Q010_E,TX_RESP_Q012_B,TX_RESP_Q012_E,TX_RESP_Q013_C,TX_RESP_Q014_C,TX_RESP_Q014_D,TX_RESP_Q015_A,TX_RESP_Q015_B,TX_RESP_Q017_B,TX_RESP_Q017_C,TX_RESP_Q036_A,TX_RESP_Q036_E,TX_RESP_Q002_B,TX_RESP_Q002_E,TX_RESP_Q016_A,TX_RESP_Q019_F,TX_RESP_Q023_F,TX_RESP_Q004_B,TX_RESP_Q004_G,TX_RESP_Q004_H,TX_RESP_Q003_I,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_MESORREGIAO,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,...,IN_EQUIP_TV,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_FAX,NU_EQUIP_TV,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_MULTIMIDIA,NU_EQUIP_FAX,NU_COMP_ALUNO,NU_FUNCIONARIOS,IN_FINAL_SEMANA,IN_EJA,IN_COMUM_FUND_AF,ID_MUNICIPIO,TX_RESP_Q040_B,TX_RESP_Q050_B,TX_RESP_Q051_D,TX_RESP_Q010_C,TX_RESP_Q037_C,TX_RESP_Q003_D,CO_MUNICIPIO,CO_DISTRITO,IN_ESGOTO_FOSSA,IN_DEPENDENCIAS_PNE,IN_AUDITORIO,IN_PATIO_COBERTO,IN_PATIO_DESCOBERTO,IN_ALOJAM_PROFESSOR,IN_LAVANDERIA,IN_EQUIP_VIDEOCASSETE,IN_EQUIP_COPIADORA,IN_EQUIP_IMPRESSORA,IN_EQUIP_SOM,NU_EQUIP_DVD,NU_EQUIP_PARABOLICA,IN_FUNDAMENTAL_CICLOS,IN_PROFISSIONALIZANTE,IN_COMUM_EJA_FUND,IN_COMUM_EJA_PROF,Valor Real Ideb,Previsão Ideb
28012631,2,6,4,0,1,0,8,1,0,0,2,1,0,0,3,0,3,3,0,0,1,3,0,5,0,7,0,0,4,2,0,0,0,3,6,1486944000000000000,1513900800000000000,2803,1,1,...,1,1,0,1,1,1,0,10,17,0,1,0,2800100,0,0,0,0,0,1,2800100,280010005,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,abaixo da média,abaixo da média
28017846,1,6,6,7,0,0,10,0,0,0,0,0,0,0,13,0,3,10,0,0,2,2,0,5,0,10,3,0,1,0,1,0,0,3,0,1494201600000000000,1519776000000000000,2803,1,1,...,1,1,0,3,2,1,0,1,55,0,0,1,2800308,1,0,2,0,5,2,2800308,280030805,0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,acima da média,acima da média
28017854,1,9,15,15,5,1,41,5,1,0,6,2,0,0,31,0,1,21,4,0,16,4,1,16,2,24,2,2,1,3,2,5,6,11,0,1491782400000000000,1519344000000000000,2803,1,1,...,1,0,0,2,0,1,0,22,81,1,0,1,2800308,1,11,0,2,17,4,2800308,280030805,0,1,1,1,1,0,0,0,1,1,1,2,0,0,0,0,0,abaixo da média,abaixo da média
28018400,1,14,3,16,0,0,32,4,1,0,2,2,0,0,28,0,6,23,1,0,5,3,1,4,0,22,2,1,8,7,5,0,0,3,0,1492387200000000000,1519862400000000000,2803,1,1,...,1,1,1,3,1,2,1,25,75,0,0,1,2800308,1,5,0,1,6,8,2800308,280030805,0,1,1,1,0,0,0,1,1,1,1,1,0,0,0,0,0,acima da média,acima da média
28018435,1,20,14,8,2,2,63,2,2,1,2,1,1,0,42,0,8,29,9,2,6,9,2,12,4,20,2,3,14,6,6,6,5,11,0,1489363200000000000,1515196800000000000,2803,1,1,...,1,0,0,4,0,6,0,26,86,1,0,1,2800308,2,13,2,0,16,14,2800308,280030805,0,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,acima da média,acima da média
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,25,15,33,4,4,51,10,3,0,5,0,0,0,63,0,5,55,6,0,11,13,3,45,12,34,2,2,15,5,6,15,16,18,2,1493683200000000000,1518739200000000000,2802,1,1,...,1,0,0,5,0,5,0,24,90,1,0,0,2807105,2,16,2,0,34,24,2807105,280710505,1,1,0,1,0,1,0,0,1,1,1,5,0,0,0,0,0,acima da média,acima da média
28009487,2,6,2,6,0,0,15,0,0,0,0,0,0,0,6,0,0,3,0,0,0,0,0,10,1,1,0,0,1,0,0,4,1,1,2,1490745600000000000,1516838400000000000,2802,1,1,...,1,1,0,2,2,2,0,8,35,1,0,1,2807105,0,5,0,1,5,4,2807105,280710505,0,1,0,1,1,0,0,0,0,1,1,1,0,0,0,0,0,acima da média,acima da média
28014839,2,3,15,13,1,0,22,2,0,0,2,1,0,0,14,0,1,15,2,0,2,3,1,14,6,15,1,0,5,2,3,4,4,5,5,1488412800000000000,1517011200000000000,2803,1,1,...,1,1,0,7,3,0,0,16,55,0,1,1,2807204,0,9,1,0,9,5,2807204,280720405,1,0,0,1,1,0,0,0,1,1,1,0,0,1,0,1,0,acima da média,acima da média
28010132,2,21,27,16,2,1,44,6,1,0,5,1,0,0,21,0,1,27,2,0,11,6,1,30,5,18,0,3,7,2,3,10,12,12,2,1488758400000000000,1516320000000000000,2802,1,1,...,1,1,0,3,2,4,0,2,53,0,0,1,2807402,2,12,2,0,18,13,2807402,280740205,1,1,0,1,1,0,0,0,1,1,1,1,1,0,0,0,0,a

In [ ]:
#dados_para_exportar.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/resultado_analise_merge_dataset_completo.csv')
#dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo.csv')
dados_para_exportar.to_csv('/content/drive/MyDrive/TCC/resultado_analise_merge_dataset_completo_SE.csv')

In [ ]:
dados_para_exportar['TX_RESP_Q005_A'].sum()

210

#### -Terceira análise: variáveis das escolas e professores

In [ ]:
columns_escolas = dataset.columns[237:404]

X_escolas = dataset[columns_escolas]

X_escolas

,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,CO_MUNICIPIO,CO_DISTRITO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_REGULAMENTACAO,IN_LOCAL_FUNC_PREDIO_ESCOLAR,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_EMPRESA,IN_LOCAL_FUNC_SOCIOEDUCATIVO,IN_LOCAL_FUNC_UNID_PRISIONAL,IN_LOCAL_FUNC_PRISIONAL_SOCIO,IN_LOCAL_FUNC_TEMPLO_IGREJA,IN_LOCAL_FUNC_CASA_PROFESSOR,IN_LOCAL_FUNC_GALPAO,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_LOCAL_FUNC_OUTROS,IN_PREDIO_COMPARTILHADO,IN_AGUA_FILTRADA,IN_AGUA_REDE_PUBLICA,IN_AGUA_POCO_ARTESIANO,IN_AGUA_CACIMBA,IN_AGUA_FONTE_RIO,IN_AGUA_INEXISTENTE,IN_ENERGIA_REDE_PUBLICA,IN_ENERGIA_GERADOR,IN_ENERGIA_OUTROS,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_REDE_PUBLICA,IN_ESGOTO_FOSSA,IN_ESGOTO_INEXISTENTE,IN_LIXO_COLETA_PERIODICA,IN_LIXO_QUEIMA,IN_LIXO_JOGA_OUTRA_AREA,IN_LIXO_RECICLA,...,IN_COMUM_PRE,IN_COMUM_FUND_AI,IN_COMUM_FUND_AF,IN_COMUM_MEDIO_MEDIO,IN_COMUM_MEDIO_INTEGRADO,IN_COMUM_MEDIO_NORMAL,IN_ESP_EXCLUSIVA_CRECHE,IN_ESP_EXCLUSIVA_PRE,IN_ESP_EXCLUSIVA_FUND_AI,IN_ESP_EXCLUSIVA_FUND_AF,IN_ESP_EXCLUSIVA_MEDIO_MEDIO,IN_ESP_EXCLUSIVA_MEDIO_INTEGR,IN_ESP_EXCLUSIVA_MEDIO_NORMAL,IN_COMUM_EJA_FUND,IN_COMUM_EJA_MEDIO,IN_COMUM_EJA_PROF,IN_ESP_EXCLUSIVA_EJA_FUND,IN_ESP_EXCLUSIVA_EJA_MEDIO,IN_ESP_EXCLUSIVA_EJA_PROF,IN_COMUM_PROF,IN_ESP_EXCLUSIVA_PROF,QtdeDocentesComLicenciatura,QtdeDocentesAtendEsp,QtdeDocentesDivSocCult,QtdeDocentesEsp,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentesSemPos,QtdeDocentesFCEM,QtdeDocentes,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf
28012631,1486944000000000000,1513900800000000000,2,2803,28007,28,2800100,280010005,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,8,0,0,4,1,0,3,0,8,0,0,7,1,0,0,8,22,8,2
28017846,1494201600000000000,1519776000000000000,2,2803,28011,28,2800308,280030805,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0,0,10,2,0,10,0,22,0,0,22,0,0,0,22,121,22,5
28017854,1491782400000000000,1519344000000000000,2,2803,28011,28,2800308,280030805,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,0,0,17,2,0,13,0,32,0,0,32,0,0,0,32,301,32,9
28018400,1492387200000000000,1519862400000000000,2,2803,28011,28,2800308,280030805,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,17,4,0,14,3,34,0,0,34,0,0,0,34,383,34,11
28018435,1489363200000000000,1515196800000000000,2,2803,28011,28,2800308,280030805,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37,0,0,24,5,0,14,0,41,0,0,41,0,0,0,41,476,41,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,1493683200000000000,1518739200000000000,2,2802,28005,28,2807105,280710505,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,46,0,0,31,3,0,18,3,51,0,0,46,5,0,0,51,611,51,11
28009487,1490745600000000000,1516838400000000000,2,2802,28005,28,2807105,280710505,2,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,0,7,1,0,6,0,13,0,0,12,1,0,0,13,176,13,13
28014839,1488412800000000000,1517011200000000000,2,2803,28008,28,2807204,280720405,2,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,...,0,1,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,16,0,0,4,0,1,11,1,16,0,1,16,0,0,1,17,198,17,11
28010132,1488758400000000000,1516320000000000000,2,2802,28005,28,2807402,280740205,2,1,1,1,1,0,0,0,0,0,0,0,0,

In [ ]:
X_escolas.drop(columns=['DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO'], axis=1, inplace=True)

X_transform = minmax(X_escolas)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 43.75 %
Naive Bayes: Acurácia do modelo: 56.25 %
Decision Tree: Acurácia do modelo: 37.5 %
SVM: Acurácia do modelo: 37.5 %
Logistic Regression: Acurácia do modelo: 37.5 %
KNN: Acurácia do modelo: 81.25 %


**Aplicando seleção de atributos (Filtragem)**

In [ ]:
X_selection = selecao_atributos(X_escolas, y, 100)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [  0   3  11  12  13  14  15  16  17  22  25  26  27  28  29  30  36  37
  38  39  48  52  53  55  56  67  71  86 102 105 111 113 114 116 117 118
 119 120 121 124 125 128 130 131 132 133 134 135 136 137 141 142 143 145
 147 148 155 159] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 50.0 %
Naive Bayes: Acurácia do modelo: 56.25 %
Decision Tree: Acurácia do modelo: 37.5 %
SVM: Acurácia do modelo: 37.5 %
Logistic Regression: Acurácia do modelo: 37.5 %
KNN: Acurácia do modelo: 81.25 %


**Aplicando seleção de atributos (Embutida)**

In [ ]:
X_embedded = embedded_based(X, X_transform, y, 100)

X_embedded

,ID_REGIAO,ID_AREA,ID_DEPENDENCIA_ADM,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q026_A,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q032_B,TX_RESP_Q032_C,TX_RESP_Q033_A,TX_RESP_Q033_C,TX_RESP_Q034_A,TX_RESP_Q034_B,TX_RESP_Q035_B,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_B,TX_RESP_Q046_B,TX_RESP_Q047_C,TX_RESP_Q048_C,TX_RESP_Q049_A,TX_RESP_Q049_B,TX_RESP_Q049_C,TX_RESP_Q050_A,TX_RESP_Q050_B,TX_RESP_Q050_C,TX_RESP_Q054_A,TX_RESP_Q054_B,TX_RESP_Q054_C,TX_RESP_Q055_B,TX_RESP_Q056_B,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q058_A,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q051_B,TX_RESP_Q051_D,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q060_A,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q006_C,TX_RESP_Q006_D,TX_RESP_Q008_E,TX_RESP_Q009_A,TX_RESP_Q011_B,TX_RESP_Q011_C,TX_RESP_Q011_D,TX_RESP_Q012_E,TX_RESP_Q013_D,TX_RESP_Q014_A
28012631,2,2,2,6,15,4,0,13,6,14,18,1,15,3,9,6,10,3,2,9,6,0,10,7,9,2,6,1,18,0,15,0,0,4,11,0,17,0,1,14,4,0,6,13,3,12,0,15,1,10,5,3,0,0,0,8,8,4,1,0,1,0,0,11,13,1,0,0,0,0
28017846,2,1,2,20,37,4,2,35,6,37,39,2,31,10,30,12,19,10,5,25,9,7,13,14,18,5,15,8,36,5,24,3,3,16,19,3,38,0,2,36,3,0,16,8,7,27,3,33,0,13,25,3,0,3,2,31,10,4,3,0,1,0,0,24,22,1,0,0,0,1
28017854,2,1,2,62,93,10,2,92,9,86,95,5,85,15,55,17,49,38,17,59,21,15,37,47,41,24,37,21,84,11,65,4,3,49,34,2,87,11,0,79,7,8,23,31,12,64,10,75,8,36,53,13,1,14,0,48,41,8,7,1,6,1,0,63,56,1,0,0,0,1
28018400,2,1,2,36,71,2,2,59,14,56,70,3,54,17,45,10,33,30,12,41,20,16,28,33,30,19,14,7,70,2,47,3,1,34,33,1,64,5,0,63,5,1,8,34,8,58,1,56,0,38,31,4,0,13,0,31,32,3,8,1,5,0,0,31,51,0,0,0,1,0
28018435,2,1,2,79,124,14,4,120,20,115,135,4,122,19,77,31,51,59,19,75,43,8,67,48,67,31,41,24,125,14,107,5,4,79,51,3,119,13,2,123,11,1,19,55,10,112,3,107,1,56,70,13,2,19,2,64,63,13,5,2,3,0,1,62,79,1,0,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,2,2,186,253,39,4,265,25,246,286,3,218,75,175,137,127,30,47,172,59,33,120,115,143,60,96,66,259,30,223,5,2,160,101,2,263,16,2,256,12,1,30,104,34,227,2,242,0,86,151,53,4,39,2,240,51,23,8,3,14,0,1,181,70,2,0,0,0,0
28009487,2,2,2,23,32,10,3,36,6,33,41,1,33,8,22,16,20,6,7,28,6,6,15,9,30,9,15,8,39,3,34,0,0,28,14,0,37,5,0,40,2,0,5,16,5,36,0,25,0,5,31,5,0,0,0,27,15,2,1,0,2,0,0,29,7,0,0,0,0,2
28014839,2,2,2,42,68,6,1,71,3,66,74,0,66,7,39,30,32,11,11,36,19,13,26,37,24,19,27,15,65,7,47,2,0,35,24,3,57,9,0,62,5,1,13,15,11,48,4,49,2,36,33,3,0,2,1,48,22,15,0,0,3,0,0,44,21,0,0,0,2,1
28010132,2,2,2,76,110,31,12,122,21,109,138,5,110,31,89,51,56,37,21,77,32,16,62,39,73,36,46,31,119,20,99,1,0,66,53,2,121,12,0,132,1,1,35,56,10,108,3,91,0,25,74,41,1,13,2,90,44,4,1,1,9,1,0,103,49,0,0,0,1,1


In [ ]:
X_transform_em = minmax(X_embedded)

X_train, X_test, y_train, y_test = train_test_split(X_transform_em, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 62.5 %
Naive Bayes: Acurácia do modelo: 62.5 %
Decision Tree: Acurácia do modelo: 43.75 %
SVM: Acurácia do modelo: 56.25 %
Logistic Regression: Acurácia do modelo: 62.5 %
KNN: Acurácia do modelo: 56.25 %


**Aplicando seleção de atributos (Wrapper)**

In [ ]:
X_wrapper = wrapper_based(X, X_transform, y, 100)

X_wrapper

,ID_REGIAO,ID_UF,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q027_B,TX_RESP_Q028_A,TX_RESP_Q028_B,TX_RESP_Q029_A,TX_RESP_Q030_A,TX_RESP_Q030_B,TX_RESP_Q044_A,TX_RESP_Q044_B,TX_RESP_Q052_A,TX_RESP_Q052_B,TX_RESP_Q018_A,TX_RESP_Q022_C,TX_RESP_Q026_A,TX_RESP_Q026_B,TX_RESP_Q026_C,TX_RESP_Q032_A,TX_RESP_Q032_B,TX_RESP_Q032_C,TX_RESP_Q033_A,TX_RESP_Q033_C,TX_RESP_Q034_A,TX_RESP_Q034_B,TX_RESP_Q035_B,TX_RESP_Q040_B,TX_RESP_Q040_C,TX_RESP_Q041_B,TX_RESP_Q041_C,TX_RESP_Q042_A,TX_RESP_Q042_B,TX_RESP_Q042_C,TX_RESP_Q046_A,...,TX_RESP_Q057_A,TX_RESP_Q057_B,TX_RESP_Q057_C,TX_RESP_Q058_A,TX_RESP_Q058_B,TX_RESP_Q058_C,TX_RESP_Q039_A,TX_RESP_Q039_B,TX_RESP_Q039_C,TX_RESP_Q039_D,TX_RESP_Q045_A,TX_RESP_Q045_B,TX_RESP_Q045_C,TX_RESP_Q051_A,TX_RESP_Q051_B,TX_RESP_Q051_D,TX_RESP_Q053_A,TX_RESP_Q053_B,TX_RESP_Q053_C,TX_RESP_Q053_D,TX_RESP_Q059_B,TX_RESP_Q060_A,TX_RESP_Q005_D,TX_RESP_Q005_E,TX_RESP_Q006_C,TX_RESP_Q006_D,TX_RESP_Q008_E,TX_RESP_Q009_A,TX_RESP_Q009_B,TX_RESP_Q011_A,TX_RESP_Q011_B,TX_RESP_Q011_C,TX_RESP_Q011_D,TX_RESP_Q011_E,TX_RESP_Q012_C,TX_RESP_Q012_D,TX_RESP_Q012_E,TX_RESP_Q013_B,TX_RESP_Q013_D,TX_RESP_Q014_A
28012631,2,28,2,2,1,12,6,15,4,0,13,6,14,18,1,15,3,9,9,17,0,6,10,3,2,9,6,0,10,7,9,2,0,3,6,1,18,0,1,1,...,3,12,0,15,1,1,10,5,3,0,11,6,1,18,0,0,8,8,3,0,0,4,1,0,1,0,0,11,6,5,13,1,0,0,1,0,0,4,0,0
28017846,2,28,1,2,1,21,20,37,4,2,35,6,37,39,2,31,10,30,10,36,1,12,19,10,5,25,9,7,13,14,18,5,1,5,15,8,36,5,0,10,...,7,27,3,33,5,0,13,25,3,0,24,12,4,36,3,2,31,10,0,0,4,4,3,0,1,0,0,24,16,18,22,1,0,0,0,0,0,14,0,1
28017854,2,28,1,2,1,39,62,93,10,2,92,9,86,95,5,85,15,55,44,76,4,17,49,38,17,59,21,15,37,47,41,24,1,13,37,21,84,11,5,17,...,12,64,10,75,16,8,36,53,13,1,69,31,1,88,14,0,48,41,7,7,20,8,7,1,6,1,0,63,37,46,56,1,0,0,1,0,0,37,0,1
28018400,2,28,1,2,1,36,36,71,2,2,59,14,56,70,3,54,17,45,27,57,3,10,33,30,12,41,20,16,28,33,30,19,1,8,14,7,70,2,0,18,...,8,58,1,56,13,0,38,31,4,0,44,25,2,60,13,0,31,32,9,0,20,3,8,1,5,0,0,31,36,22,51,0,0,0,4,0,0,40,1,0
28018435,2,28,1,2,1,56,79,124,14,4,120,20,115,135,4,122,19,77,58,105,6,31,51,59,19,75,43,8,67,48,67,31,2,18,41,24,125,14,2,19,...,10,112,3,107,23,1,56,70,13,2,71,59,5,120,19,2,64,63,12,1,18,13,5,2,3,0,1,62,72,58,79,1,0,0,4,0,0,59,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,28,2,2,1,106,186,253,39,4,265,25,246,286,3,218,75,175,117,247,15,137,127,30,47,172,59,33,120,115,143,60,2,19,96,66,259,30,4,38,...,34,227,2,242,34,0,86,151,53,4,234,55,3,245,39,2,240,51,2,0,40,23,8,3,14,0,1,181,111,222,70,2,0,0,1,0,0,98,0,0
28009487,2,28,2,2,2,18,23,32,10,3,36,6,33,41,1,33,8,22,20,35,1,16,20,6,7,28,6,6,15,9,30,9,0,1,15,8,39,3,0,8,...,5,36,0,25,16,0,5,31,5,0,37,5,0,42,0,0,27,15,0,0,19,2,1,0,2,0,0,29,13,35,7,0,0,0,1,0,0,10,0,2
28014839,2,28,2,2,1,31,42,68,6,1,71,3,66,74,0,66,7,39,33,64,4,30,32,11,11,36,19,13,26,37,24,19,0,2,27,15,65,7,1,12,...,11,48,4,49,15,2,36,33,3,0,52,18,1,69,2,1,48,22,3,0,11,15,0,0,3,0,0,44,26,54,21,0,0,0,0,0,0,13,2,1
28010132,2,28,2,2,1,62,76,110,31,12,122,21,109,138,5,110,31,89,48,101,6,51,56,37,21,77,32,16,62,39,73,36,2,6,46,31,119,20,2,18,...,10,108,3,91,34,0,25,74,41,1,91,46,2,122,13,2,90,44,6,1,7,4,1,1,9,1,0,103,38,94,49,0,0,0,1,0,0,33,1,1


In [ ]:
X_transform_w = minmax(X_wrapper)

X_train, X_test, y_train, y_test = train_test_split(X_transform_w, y, random_state=50, test_size=0.2)

In [ ]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [ ]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 68.75 %
Naive Bayes: Acurácia do modelo: 56.25 %
Decision Tree: Acurácia do modelo: 68.75 %
SVM: Acurácia do modelo: 68.75 %
Logistic Regression: Acurácia do modelo: 62.5 %
KNN: Acurácia do modelo: 56.25 %
